# Install Networkx library

In [1]:
!pip install networkx

# Import appropraite libraries

In [2]:
import networkx as nx
import random
import matplotlib.pyplot as plt
import math
import ast
import numpy as np
import pandas as pd
import random
from collections import Counter
from random import triangular
import warnings
warnings.filterwarnings('ignore')

# Generate distance for national topology

In [3]:
# Create a function to give 16 links with different lenghts paying attention to average = 150 and maximum=400
def generate_numbers_with_average(average = 150, maximum =400, num_distance=16):
    mode = average
    if mode < 140 or mode > 160:
        raise ValueError(f"Average {average} cannot be satisfied within the range")

    distnace_list = [int(triangular(0, maximum, mode)) for _ in range(num_distance)]
    average_distance = sum(distnace_list) / num_distance
    print('average_lenght: ',average_distance)
    return distnace_list

distnace_list = generate_numbers_with_average()
print('links with different lenghts: ',distnace_list)

average_lenght:  170.625
links with different lenghts:  [175, 158, 212, 114, 324, 110, 26, 100, 280, 100, 64, 272, 318, 174, 149, 154]


# Generate distance for continental topology

In [4]:
# Create a function to give 20 links with different lenghts paying attention to average = 300 and maximum=1000

def generate_numbers_with_average(average = 300, maximum =1000, num_distance=20):
    mode = average
    if mode < 250 or mode > 350:
        raise ValueError(f"Average {average} cannot be satisfied within the range")

    distnace_list = [int(triangular(0, maximum, mode)) for _ in range(num_distance)]
    average_distance = sum(distnace_list) / num_distance
    print('average_lenght: ',average_distance)
    return distnace_list

distnace_list_cont = generate_numbers_with_average()
print('links with different lenghts: ',distnace_list_cont)

average_lenght:  361.7
links with different lenghts:  [362, 587, 322, 289, 830, 196, 414, 466, 527, 187, 284, 214, 673, 225, 202, 288, 57, 492, 289, 330]


# 1: Top_down_national_with_grooming

In [5]:
!gdown 1VlFUsmVVPDcU5ksqNtKCGxrOx8FJWE73 # Download random_demands.csv file genereated by random_demands.py

Downloading...
From: https://drive.google.com/uc?id=1VlFUsmVVPDcU5ksqNtKCGxrOx8FJWE73
To: /content/random_demands.csv
100% 220k/220k [00:00<00:00, 66.3MB/s]


In [6]:
class Top_down_national_with_grooming():

    def __init__(self):
        """Setup initial parameter while creating class"""

        self.update_demand = False
        self.step = 0
        self.info_dict = {}  # to store values

        self.table = pd.read_csv('/content/random_demands.csv')
        self.initial_number = 300
        self.generated_demands = self.table.iloc[:self.initial_number]

        # For Grooming
    def setup_graph(self):
      global distnace_list
    # Create a complete graph with 8 nodes
      self.G = nx.complete_graph(8)

      # List all possible edges
      all_edges = list(self.G.edges())

      # Remove edges until the number of edges reaches
      while len(self.G.edges()) > 16:
          # Remove the last edge from the list
          edge_to_remove = all_edges.pop()

          # Remove the chosen edge
          self.G.remove_edge(*edge_to_remove)

      # Print the edges of the resulting graph
      self.node_list = self.G.nodes()
      self.edge_list = self.G.edges()

      print('links',len(self.edge_list))

      for index, edge in enumerate(self.G.edges()):
        weight = distnace_list[index]
        self.G.add_edge(*edge,weight=weight,channels=96)

      # Fixed variabels
      self.total_bandwidth = 8*500*96 + 8*400*96
      self.total_channels = 16*96



    def add_new_demand(self):
        """ad 50 another demands to random_demands table"""

        self.initial_number += 50
        self.generated_demands = self.table.iloc[:self.initial_number]
        self.grooming()
    ##############################################################################


    def draw_graph(self,title):
        """draw_graph function"""

        nx.draw(self.G,with_labels=True,node_color=self.node_color,pos=self.node_positions)
        self.edge_labels = nx.get_edge_attributes(self.G, 'channels')
        plt.title(title, fontweight="bold")
        nx.draw_networkx_edge_labels(self.G, pos=self.node_positions, edge_labels=self.edge_labels)
        plt.show()

    ##############################################################################
    def shortest_paths(self,source,destination):
        """Find all shortest paths for each source and destination"""

        self.source = source
        self.destination = destination
        paths_dict ={}
        # We use simple paths at the beginning
        all_simple_paths = list(nx.all_simple_paths(self.G, self.source, self.destination)) # l

        index = 1
        for path in all_simple_paths:
          weight = sum(self.G[path[i]][path[i+1]]['weight'] for i in range(len(path)-1))
          paths_dict[f'path_{index}'] = {'path':path,'weight':weight}
          index+=1

        # Sort all simple paths based on their weight to get shortest_paths
        self.sorted_paths = dict(sorted(paths_dict.items(), key=lambda item: item[1]['weight']))

    ##############################################################################
    def channel_attribute(self,source,destination):
        """Add channel attribute to each  link of chosen shortest_path"""

        self.source = source
        self.destination = destination

        # call shortest_path
        self.shortest_paths(self.source,self.destination)
        self.sh_p = self.sorted_paths

        self.shortest_paths_with_channels ={}
        for shortest_path_key in self.sh_p.keys(): # loop for all shortest_paths
            chosen_shortest_path = self.sh_p[shortest_path_key]
            no_links_here =len(chosen_shortest_path['path'])-1

            link_attrs ={}
            for i in range(no_links_here):
              free_channles = self.G[chosen_shortest_path['path'][i]][chosen_shortest_path['path'][i+1]]['channels']
              link_attrs[(chosen_shortest_path['path'][i], chosen_shortest_path['path'][i+1])]={"free_channels":free_channles }


            self.shortest_paths_with_channels[shortest_path_key] = link_attrs



    def grooming(self):

      self.new_table = pd.DataFrame({'no_demands':[],'data_rate':[],'source':[],'destination':[]})
      for source in self.node_list:
        for destination in self.node_list:
          if source != destination:
            for data_rate  in [100,400] :
              result = self.generated_demands[((self.generated_demands.source==source) & (self.generated_demands.destination==destination) & (self.generated_demands.data_rate==data_rate))]
              no_demands = result.shape[0]
              self.new_table.loc[len(self.new_table)] = [no_demands,data_rate,source,destination]

      self.generated_demands = self.new_table
      print('I am in grooming')

    ##############################################################################
    def routing(self):
        """Routing part"""

        # setup_graph
        self.setup_graph()


        # check if there are initial demands in the system or not. If so, we do not need to call initial_random_demand
        if self.update_demand ==False:
          self.initial_number = 300
          self.generated_demands = self.table.iloc[:self.initial_number]
          self.grooming()

        else:
          pass

        self.input_traffic_list = []
        self.required_channels_list =[]
        self.blocked_demand_list = []

        # iterate over all demands
        #for demand in self.generated_demands:
        for index in self.generated_demands.index:
            demand = self.generated_demands.iloc[index]


            no_demand = demand['no_demands']
            data_rate = demand['data_rate']
            source = demand['source']
            destination = demand['destination']

            print(f'\t\t\t\t\t ******  Demand  at this level is [{no_demand},{data_rate},({source},{destination})]  ******')

            # calculate input traffic
            self.input_traffic = no_demand*data_rate

            # call channel_attribute to first find short paths between source and destination,
            # then set channel attributes to these paths
            self.channel_attribute(source=source,destination=destination)

            # iterate over all short paths available for the demand
            for shortest_path_name in self.shortest_paths_with_channels.keys():


                # Chose one shortest path
                chosen_shortest_path= self.shortest_paths_with_channels[shortest_path_name] # loop for all shortest_paths
                print('\nFor this demand, chosed shortest path is--->',chosen_shortest_path)



                positive_counter = 0
                req_channles_dict = {}
                # calculate distance

                self.total_distance = 0
                for link in chosen_shortest_path.keys():
                  self.total_distance += self.G[link[0]][link[1]]['weight']

                if self.total_distance <=1300:
                  print(f'Set data_rate to 500, since total_distance <= 1300')
                  self.data_rate=500

                elif 1300 < self.total_distance <= 2500:
                  print(f'Set data_rate to 400, since 1300 < total_distan <= 2500')
                  self.data_rate=400
                else:
                  self.next_shortest_path = True

                # iterate over all links of chosen the shortest path
                for link in chosen_shortest_path.keys():

                      self.free_channel = chosen_shortest_path[link]['free_channels']


                      # add input traffic of this link to input_traffic_list to use in FR, AVO and ...
                      self.input_traffic_list.append(self.input_traffic)

                      # calculate the number WDM trails(required_channels) and it to required_channels_list
                      self.required_channels = math.ceil(self.input_traffic / self.data_rate)
                      self.required_channels_list.append(self.required_channels)
                      req_channles_dict[link] = self.required_channels



                      if self.free_channel>=self.required_channels:
                            positive_counter+=1

                print(f'Required channels(WDMS) for chosed path is:{(self.input_traffic / self.data_rate)}-->{self.required_channels}')

                # If positive_counter == number of links in a chosen short path, it means all links have appropriate
                # capacity to pass the demand
                if positive_counter == len(chosen_shortest_path.keys()):

                    # pass required_channels from each link of chosen path and update free channels
                    for link in chosen_shortest_path.keys():
                            self.required_channels = req_channles_dict[link]
                            self.shortest_paths_with_channels[shortest_path_name][link]['free_channels']-= self.required_channels
                            self.G[link[0]][link[1]]['channels'] -= self.required_channels
                    print('\nUpdataed chosed_shortest_path is: ',self.shortest_paths_with_channels[shortest_path_name])
                    print('='*200)


                    # Here, we use break, since the chosen pass could completely pass the demand and there is no need
                    # to check other short paths for this demand
                    break


                # If positive_counter != number of links in a chosen short path, it means some links do not
                # have appropriate capacity to pass the demand. So, we need to find bottleneck of that path
                # and pass demand == bottleneck from that chosen path and reserve the rest for next chosen_path
                else:

                    # Find bottleneck. First initial bottleneck is equal to 96 * 400
                    bottleneck =96*400

                    # look at each link . If capacity of that link < bottleneck, we consider it as the new_bottleneck
                    for link in chosen_shortest_path.keys():
                            current_bottleneck = self.shortest_paths_with_channels[shortest_path_name][link]['free_channels'] * self.data_rate

                            if current_bottleneck < bottleneck:
                                bottleneck = current_bottleneck

                    # This part remove some input traffic from list in previous
                    for link in chosen_shortest_path.keys() :
                            self.input_traffic_list.pop()
                            self.required_channels_list.pop()

                    # Iterate over each link to see the number of channels we can pass and vice versa.
                    for link in chosen_shortest_path.keys():
                            gbit_able_to_pass = bottleneck
                            gbit_not_able_to_pass = self.input_traffic - bottleneck

                            self.channel_able_to_pass = math.ceil(gbit_able_to_pass / self.data_rate)

                            # For replacing the old input traffic with new bottleneck as new traffic for calculating FR and ...
                            self.input_traffic_list.append(bottleneck)
                            self.required_channels_list.append(self.channel_able_to_pass)
                            if req_channles_dict[link]>= self.shortest_paths_with_channels[shortest_path_name][link]['free_channels']:
                                channel_not_able_to_pass = req_channles_dict[link] - self.channel_able_to_pass
                            else:
                                channel_not_able_to_pass = 0

                            print(f'But for this link({link}) we can pass ({self.channel_able_to_pass}) channels and we must reserve ({channel_not_able_to_pass}) for the next shortest_path')

                            if channel_not_able_to_pass>0:
                                req_channles_dict[link] = channel_not_able_to_pass
                                self.shortest_paths_with_channels[shortest_path_name][link]['free_channels']= 0
                                self.G[link[0]][link[1]]['channels'] = 0

                            elif channel_not_able_to_pass==0:
                                req_channles_dict[link] = 0
                                self.shortest_paths_with_channels[shortest_path_name][link]['free_channels']-= self.channel_able_to_pass
                                self.G[link[0]][link[1]]['channels'] -= self.channel_able_to_pass





                    print('\nPrevious shortest_path has been updated-->',self.shortest_paths_with_channels[shortest_path_name])
                    print('\nBut we have still some required channels. So, we go through another shortest path: ')
                    self.channel_attribute(source=source,destination=destination)# call channel to update free channels
                    self.input_traffic = gbit_not_able_to_pass # For other part


            #Checking block
            # We set minor block which means one shortest path can be block not all shortest paths
            self.minor_block = 0
            for shortest_path_name in self.shortest_paths_with_channels.keys():
                chosen_shortest_path= self.shortest_paths_with_channels[shortest_path_name] # loop for all shortest_paths
                for link in chosen_shortest_path.keys():
                    if chosen_shortest_path[link]['free_channels']==0:
                        self.minor_block +=1
                        break

                # if number of minor block == number of all shortest_paths, means all paths do not have the capacity for
                # passing the demand, and we have blocking problem
            if self.minor_block == len(self.shortest_paths_with_channels.keys()):
                print('Blocking will happen here, and no_of_blocked_demand is: ',no_demand)
                self.blocked_demand_list.append(no_demand)

            else:
                self.blocked_demand_list.append(0)

          ##########################################################################


        # Draw graph for the demand
        #self.draw_graph(title=f'step {self.step}')

        # Calculate all parameters we need for the demand
        self.calculations()

        print('='*200)
        print('50 new random demand added')
        print('='*200)
        # Add another 50 random demands to the demands, avaialble in system
        self.updating()



    ##############################################################################
    def updating(self):
        """ Check Blocking < 1 or not, if so, program will be stopped. Otherwise it addes 50 random demands """

        if self.BR<1:

                self.step +=1
                self.update_demand =True
                self.add_new_demand()
                self.routing()


        else:
          print(f'The network has been stopped because of BlooCking rate {self.BR}')
    def job(self):
      self.AVO = self.FR + 0.05 * self.FR
    ##############################################################################
    def calculations(self):
        """Calculate FR, AVO, BR"""

        self.total_no_of_demands = 0
        self.total_no_of_demands = self.step * 50 + 300
        print(f'So far, total_no_of_demands is :{self.total_no_of_demands}')

        self.no_of_blocked_demand = sum(self.blocked_demand_list)
        self.used_bandwidth = sum(self.input_traffic_list)
        self.used_channels = sum(self.required_channels_list)
        self.FR = (self.used_bandwidth / self.total_bandwidth)*100
        self.AVO = (self.used_channels / self.total_channels)*100
        self.BR = (self.no_of_blocked_demand / self.total_no_of_demands)*100
        #self.job()
        #####
        print(f'used_bandwidth: {self.used_bandwidth } -->{self.input_traffic_list}')
        print(f'used channels: {self.used_channels} --->{self.required_channels_list}')
        print(f'total_channels: {self.total_channels}')
        print('Filling Ratio is: ',self.FR)
        print('Average Fiber Ocupation: ',self.AVO)
        print('Blocking Rate: ',self.BR)
        self.info_dict[f'step {self.step}'] = {'FR':self.FR,'AVO':self.AVO,'BR':self.BR}


    def store_values(self):
        """Return values we need"""

        return self.info_dict, self.generated_demands

In [ ]:
CND = Top_down_national_with_grooming()
route = CND.routing()

In [8]:
values,generated_demands = CND.store_values()
Top_down_national_with_grooming_table = pd.DataFrame(values)
Top_down_national_with_grooming_table.to_csv('Top_down_national_with_grooming_table.csv')
Top_down_national_with_grooming_table

,step 0,step 1,step 2,step 3,step 4,step 5,step 6,step 7,step 8,step 9,...,step 14,step 15,step 16,step 17,step 18,step 19,step 20,step 21,step 22,step 23
FR,17.824074,20.196759,23.061343,25.405093,28.616898,31.264468,33.998843,36.877894,39.453125,42.317708,...,56.741898,60.069444,62.86169,65.205440,68.938079,71.166088,74.392361,77.213542,80.830440,78.399884
AVO,20.833333,22.916667,25.781250,27.669271,30.664062,33.007812,34.895833,37.239583,39.518229,42.317708,...,55.013021,58.203125,60.93750,62.825521,66.276042,68.229167,71.484375,74.414062,77.669271,75.325521
BR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.448276


# 2: Top_down_national_without_grooming

In [9]:

class Top_down_national_without_grooming():

    def __init__(self):
        """Setup initial parameter while creating class"""

        self.update_demand = False
        self.step = 0
        self.info_dict = {}  # to store values

        self.table = pd.read_csv('/content/random_demands.csv')
        self.initial_number = 300
        self.generated_demands = self.table.iloc[:self.initial_number]

        # For Grooming
    def setup_graph(self):
      global distnace_list
    # Create a complete graph with 8 nodes
      self.G = nx.complete_graph(8)

      # List all possible edges
      all_edges = list(self.G.edges())

      # Remove edges until the number of edges reaches
      while len(self.G.edges()) > 16:
          # Remove the last edge from the list
          edge_to_remove = all_edges.pop()

          # Remove the chosen edge
          self.G.remove_edge(*edge_to_remove)

      # Print the edges of the resulting graph
      self.node_list = self.G.nodes()
      self.edge_list = self.G.edges()

      print('links',len(self.edge_list))

      for index, edge in enumerate(self.G.edges()):
        weight = distnace_list[index]
        self.G.add_edge(*edge,weight=weight,channels=96)

      # Fixed variabels
      self.total_bandwidth = 8*500*96 + 8*400*96
      self.total_channels = 16*96
    ##############################################################################


    def add_new_demand(self):
        """ad 50 another demands to random_demands table"""

        self.initial_number += 50
        self.generated_demands = self.table.iloc[:self.initial_number]
        #self.grooming()
    ##############################################################################


    ##############################################################################
    def draw_graph(self,title):
        """draw_graph function"""

        nx.draw(self.G,with_labels=True,node_color=self.node_color,pos=self.node_positions)
        self.edge_labels = nx.get_edge_attributes(self.G, 'channels')
        plt.title(title, fontweight="bold")
        nx.draw_networkx_edge_labels(self.G, pos=self.node_positions, edge_labels=self.edge_labels)
        plt.show()

    ##############################################################################
    def shortest_paths(self,source,destination):
        """Find all shortest paths for each source and destination"""

        self.source = source
        self.destination = destination
        paths_dict ={}
        # We use simple paths at the beginning
        all_simple_paths = list(nx.all_simple_paths(self.G, self.source, self.destination)) # l

        index = 1
        for path in all_simple_paths:
          weight = sum(self.G[path[i]][path[i+1]]['weight'] for i in range(len(path)-1))
          paths_dict[f'path_{index}'] = {'path':path,'weight':weight}
          index+=1

        # Sort all simple paths based on their weight to get shortest_paths
        self.sorted_paths = dict(sorted(paths_dict.items(), key=lambda item: item[1]['weight']))

    ##############################################################################
    def channel_attribute(self,source,destination):
        """Add channel attribute to each  link of chosen shortest_path"""

        self.source = source
        self.destination = destination

        # call shortest_path
        self.shortest_paths(self.source,self.destination)
        self.sh_p = self.sorted_paths

        self.shortest_paths_with_channels ={}
        for shortest_path_key in self.sh_p.keys(): # loop for all shortest_paths
            chosen_shortest_path = self.sh_p[shortest_path_key]
            no_links_here =len(chosen_shortest_path['path'])-1

            link_attrs ={}
            for i in range(no_links_here):
              free_channles = self.G[chosen_shortest_path['path'][i]][chosen_shortest_path['path'][i+1]]['channels']
              link_attrs[(chosen_shortest_path['path'][i], chosen_shortest_path['path'][i+1])]={"free_channels":free_channles }


            self.shortest_paths_with_channels[shortest_path_key] = link_attrs

        # print('All shortest paths  with added channels attribute:')


        # for path in self.shortest_paths_with_channels.keys():
        #   print(self.shortest_paths_with_channels[path])

    def grooming(self):

      self.new_table = pd.DataFrame({'no_demands':[],'data_rate':[],'source':[],'destination':[]})
      for source in self.node_list:
        for destination in self.node_list:
          if source != destination:
            for data_rate  in [100,400] :
              result = self.generated_demands[((self.generated_demands.source==source) & (self.generated_demands.destination==destination) & (self.generated_demands.data_rate==data_rate))]
              no_demands = result.shape[0]
              self.new_table.loc[len(self.new_table)] = [no_demands,data_rate,source,destination]

      self.generated_demands = self.new_table
      print('I am in grooming')

    ##############################################################################
    def routing(self):
        """Routing part"""

        # setup_graph
        self.setup_graph()


        # check if there are initial demands in the system or not. If so, we do not need to call initial_random_demand
        if self.update_demand ==False:
          self.initial_number = 300
          self.generated_demands = self.table.iloc[:self.initial_number]
          self.grooming()

        else:
          pass

        self.input_traffic_list = []
        self.required_channels_list =[]
        self.blocked_demand_list = []

        # iterate over all demands
        #for demand in self.generated_demands:
        for index in self.generated_demands.index:
            demand = self.generated_demands.iloc[index]


            no_demand = demand['no_demands']
            data_rate = demand['data_rate']
            source = demand['source']
            destination = demand['destination']

            print(f'\t\t\t\t\t ******  Demand  at this level is [{no_demand},{data_rate},({source},{destination})]  ******')

            # calculate input traffic
            self.input_traffic = no_demand*data_rate

            # call channel_attribute to first find short paths between source and destination,
            # then set channel attributes to these paths
            self.channel_attribute(source=source,destination=destination)

            # iterate over all short paths available for the demand
            for shortest_path_name in self.shortest_paths_with_channels.keys():


                # Chose one shortest path
                chosen_shortest_path= self.shortest_paths_with_channels[shortest_path_name] # loop for all shortest_paths
                print('\nFor this demand, chosed shortest path is--->',chosen_shortest_path)



                positive_counter = 0
                req_channles_dict = {}
                # calculate distance

                self.total_distance = 0
                for link in chosen_shortest_path.keys():
                  self.total_distance += self.G[link[0]][link[1]]['weight']

                if self.total_distance <=1300:
                  print(f'Set data_rate to 500, since total_distance <= 1300')
                  self.data_rate=500

                elif 1300 < self.total_distance <= 2500:
                  print(f'Set data_rate to 400, since 1300 < total_distan <= 2500')
                  self.data_rate=400
                else:
                  self.next_shortest_path = True

                # iterate over all links of chosen the shortest path
                for link in chosen_shortest_path.keys():

                      self.free_channel = chosen_shortest_path[link]['free_channels']


                      # add input traffic of this link to input_traffic_list to use in FR, AVO and ...
                      self.input_traffic_list.append(self.input_traffic)

                      # calculate the number WDM trails(required_channels) and it to required_channels_list
                      self.required_channels = math.ceil(self.input_traffic / self.data_rate)
                      self.required_channels_list.append(self.required_channels)
                      req_channles_dict[link] = self.required_channels



                      if self.free_channel>=self.required_channels:
                            positive_counter+=1

                print(f'Required channels(WDMS) for chosed path is:{(self.input_traffic / self.data_rate)}-->{self.required_channels}')

                # If positive_counter == number of links in a chosen short path, it means all links have appropriate
                # capacity to pass the demand
                if positive_counter == len(chosen_shortest_path.keys()):

                    # pass required_channels from each link of chosen path and update free channels
                    for link in chosen_shortest_path.keys():
                            self.required_channels = req_channles_dict[link]
                            self.shortest_paths_with_channels[shortest_path_name][link]['free_channels']-= self.required_channels
                            self.G[link[0]][link[1]]['channels'] -= self.required_channels
                    print('\nUpdataed chosed_shortest_path is: ',self.shortest_paths_with_channels[shortest_path_name])
                    print('='*200)


                    # Here, we use break, since the chosen pass could completely pass the demand and there is no need
                    # to check other short paths for this demand
                    break


                # If positive_counter != number of links in a chosen short path, it means some links do not
                # have appropriate capacity to pass the demand. So, we need to find bottleneck of that path
                # and pass demand == bottleneck from that chosen path and reserve the rest for next chosen_path
                else:

                    # Find bottleneck. First initial bottleneck is equal to 96 * 400
                    bottleneck =96*400

                    # look at each link . If capacity of that link < bottleneck, we consider it as the new_bottleneck
                    for link in chosen_shortest_path.keys():
                            current_bottleneck = self.shortest_paths_with_channels[shortest_path_name][link]['free_channels'] * self.data_rate

                            if current_bottleneck < bottleneck:
                                bottleneck = current_bottleneck

                    # This part remove some input traffic from list in previous
                    for link in chosen_shortest_path.keys() :
                            self.input_traffic_list.pop()
                            self.required_channels_list.pop()

                    # Iterate over each link to see the number of channels we can pass and vice versa.
                    for link in chosen_shortest_path.keys():
                            gbit_able_to_pass = bottleneck
                            gbit_not_able_to_pass = self.input_traffic - bottleneck

                            self.channel_able_to_pass = math.ceil(gbit_able_to_pass / self.data_rate)

                            # For replacing the old input traffic with new bottleneck as new traffic for calculating FR and ...
                            self.input_traffic_list.append(bottleneck)
                            self.required_channels_list.append(self.channel_able_to_pass)
                            if req_channles_dict[link]>= self.shortest_paths_with_channels[shortest_path_name][link]['free_channels']:
                                channel_not_able_to_pass = req_channles_dict[link] - self.channel_able_to_pass
                            else:
                                channel_not_able_to_pass = 0

                            print(f'But for this link({link}) we can pass ({self.channel_able_to_pass}) channels and we must reserve ({channel_not_able_to_pass}) for the next shortest_path')

                            if channel_not_able_to_pass>0:
                                req_channles_dict[link] = channel_not_able_to_pass
                                self.shortest_paths_with_channels[shortest_path_name][link]['free_channels']= 0
                                self.G[link[0]][link[1]]['channels'] = 0

                            elif channel_not_able_to_pass==0:
                                req_channles_dict[link] = 0
                                self.shortest_paths_with_channels[shortest_path_name][link]['free_channels']-= self.channel_able_to_pass
                                self.G[link[0]][link[1]]['channels'] -= self.channel_able_to_pass





                    print('\nPrevious shortest_path has been updated-->',self.shortest_paths_with_channels[shortest_path_name])
                    print('\nBut we have still some required channels. So, we go through another shortest path: ')
                    self.channel_attribute(source=source,destination=destination)# call channel to update free channels
                    self.input_traffic = gbit_not_able_to_pass # For other part


            #Checking block
            # We set minor block which means one shortest path can be block not all shortest paths
            self.minor_block = 0
            for shortest_path_name in self.shortest_paths_with_channels.keys():
                chosen_shortest_path= self.shortest_paths_with_channels[shortest_path_name] # loop for all shortest_paths
                for link in chosen_shortest_path.keys():
                    if chosen_shortest_path[link]['free_channels']==0:
                        self.minor_block +=1
                        break

                # if number of minor block == number of all shortest_paths, means all paths do not have the capacity for
                # passing the demand, and we have blocking problem
            if self.minor_block == len(self.shortest_paths_with_channels.keys()):
                print('Blocking will happen here, and no_of_blocked_demand is: ',no_demand)
                self.blocked_demand_list.append(no_demand)

            else:
                self.blocked_demand_list.append(0)

          ##########################################################################


        # Draw graph for the demand
        #self.draw_graph(title=f'step {self.step}')

        # Calculate all parameters we need for the demand
        self.calculations()

        print('='*200)
        print('50 new random demand added')
        print('='*200)
        # Add another 50 random demands to the demands, avaialble in system
        self.updating()



    ##############################################################################
    def updating(self):
        """ Check Blocking < 1 or not, if so, program will be stopped. Otherwise it addes 50 random demands """

        if self.BR<1:

                self.step +=1
                self.update_demand =True
                self.add_new_demand()
                self.routing()


        else:
          print(f'The network has been stopped because of BlooCking rate {self.BR}')
    def job(self):
      self.AVO = self.FR + 0.05 * self.FR
    ##############################################################################
    def calculations(self):
        """Calculate FR, AVO, BR"""

        self.total_no_of_demands = 0
        self.total_no_of_demands = self.step * 50 + 300
        print(f'So far, total_no_of_demands is :{self.total_no_of_demands}')

        self.no_of_blocked_demand = sum(self.blocked_demand_list)
        self.used_bandwidth = sum(self.input_traffic_list)
        self.used_channels = sum(self.required_channels_list)
        self.FR = (self.used_bandwidth / self.total_bandwidth)*100
        self.AVO = (self.used_channels / self.total_channels)*100
        self.BR = (self.no_of_blocked_demand / self.total_no_of_demands)*100
        #self.job()
        #####
        print(f'used_bandwidth: {self.used_bandwidth } -->{self.input_traffic_list}')
        print(f'used channels: {self.used_channels} --->{self.required_channels_list}')
        print(f'total_channels: {self.total_channels}')
        print('Filling Ratio is: ',self.FR)
        print('Average Fiber Ocupation: ',self.AVO)
        print('Blocking Rate: ',self.BR)
        self.info_dict[f'step {self.step}'] = {'FR':self.FR,'AVO':self.AVO,'BR':self.BR}


    def store_values(self):
        """Return values we need"""

        return self.info_dict, self.generated_demands

In [ ]:
CND = Top_down_national_without_grooming()
route = CND.routing()

In [11]:
values,generated_demands = CND.store_values()
Top_down_national_without_grooming_table = pd.DataFrame(values)
Top_down_national_without_grooming_table.to_csv('Top_down_national_without_grooming_table.csv')
Top_down_national_without_grooming_table

,step 0,step 1,step 2,step 3,step 4,step 5,step 6,step 7,step 8,step 9,step 10
FR,17.824074,20.196759,23.075810,25.419560,28.631366,31.394676,34.563079,37.586806,40.407986,43.576389,45.442708
AVO,20.833333,36.002604,40.950521,46.028646,51.302083,56.315104,62.565104,68.359375,74.023438,79.296875,82.812500
BR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.133333,2.500000


# 3: Top_down_continental_with_grooming

In [12]:

class Top_down_continental_with_grooming():

    def __init__(self):
        """Setup initial parameter while creating class"""

        self.update_demand = False
        self.step = 0
        self.info_dict = {}  # to store values

        self.table = pd.read_csv('/content/random_demands.csv')
        self.initial_number = 300
        self.generated_demands = self.table.iloc[:self.initial_number]

        # For Grooming
    def setup_graph(self):

      global distnace_list_cont
      # Create a complete graph with 8 nodes
      self.G = nx.complete_graph(8)

      # List all possible edges
      all_edges = list(self.G.edges())

      # Remove edges until the number of edges reaches
      while len(self.G.edges()) > 20:
          # Remove the last edge from the list
          edge_to_remove = all_edges.pop()

          # Remove the chosen edge
          self.G.remove_edge(*edge_to_remove)

      # Print the edges of the resulting graph
      self.node_list = self.G.nodes()
      self.edge_list = self.G.edges()

      print('links',len(self.edge_list))

      for index, edge in enumerate(self.G.edges()):
        weight = distnace_list_cont[index]
        self.G.add_edge(*edge,weight=weight,channels=96)

      # Fixed variabels
      self.total_bandwidth = 10*500*96 + 10*400*96
      self.total_channels = 20*96
    ##############################################################################



    def add_new_demand(self):
        """ad 50 another demands to random_demands table"""

        self.initial_number += 50
        self.generated_demands = self.table.iloc[:self.initial_number]
        self.grooming()
    ##############################################################################


    ##############################################################################
    def draw_graph(self,title):
        """draw_graph function"""

        nx.draw(self.G,with_labels=True,node_color=self.node_color,pos=self.node_positions)
        self.edge_labels = nx.get_edge_attributes(self.G, 'channels')
        plt.title(title, fontweight="bold")
        nx.draw_networkx_edge_labels(self.G, pos=self.node_positions, edge_labels=self.edge_labels)
        plt.show()

    ##############################################################################
    def shortest_paths(self,source,destination):
        """Find all shortest paths for each source and destination"""

        self.source = source
        self.destination = destination
        paths_dict ={}
        # We use simple paths at the beginning
        all_simple_paths = list(nx.all_simple_paths(self.G, self.source, self.destination)) # l

        index = 1
        for path in all_simple_paths:
          weight = sum(self.G[path[i]][path[i+1]]['weight'] for i in range(len(path)-1))
          paths_dict[f'path_{index}'] = {'path':path,'weight':weight}
          index+=1

        # Sort all simple paths based on their weight to get shortest_paths
        self.sorted_paths = dict(sorted(paths_dict.items(), key=lambda item: item[1]['weight']))

    ##############################################################################
    def channel_attribute(self,source,destination):
        """Add channel attribute to each  link of chosen shortest_path"""

        self.source = source
        self.destination = destination

        # call shortest_path
        self.shortest_paths(self.source,self.destination)
        self.sh_p = self.sorted_paths

        self.shortest_paths_with_channels ={}
        for shortest_path_key in self.sh_p.keys(): # loop for all shortest_paths
            chosen_shortest_path = self.sh_p[shortest_path_key]
            no_links_here =len(chosen_shortest_path['path'])-1

            link_attrs ={}
            for i in range(no_links_here):
              free_channles = self.G[chosen_shortest_path['path'][i]][chosen_shortest_path['path'][i+1]]['channels']
              link_attrs[(chosen_shortest_path['path'][i], chosen_shortest_path['path'][i+1])]={"free_channels":free_channles }


            self.shortest_paths_with_channels[shortest_path_key] = link_attrs

        # print('All shortest paths  with added channels attribute:')


        # for path in self.shortest_paths_with_channels.keys():
        #   print(self.shortest_paths_with_channels[path])

    def grooming(self):

      self.new_table = pd.DataFrame({'no_demands':[],'data_rate':[],'source':[],'destination':[]})
      for source in self.node_list:
        for destination in self.node_list:
          if source != destination:
            for data_rate  in [100,400] :
              result = self.generated_demands[((self.generated_demands.source==source) & (self.generated_demands.destination==destination) & (self.generated_demands.data_rate==data_rate))]
              no_demands = result.shape[0]
              self.new_table.loc[len(self.new_table)] = [no_demands,data_rate,source,destination]

      self.generated_demands = self.new_table
      print('I am in grooming')

    ##############################################################################
    def routing(self):
        """Routing part"""

        # setup_graph
        self.setup_graph()


        # check if there are initial demands in the system or not. If so, we do not need to call initial_random_demand
        if self.update_demand ==False:
          self.initial_number = 300
          self.generated_demands = self.table.iloc[:self.initial_number]
          self.grooming()

        else:
          pass

        self.input_traffic_list = []
        self.required_channels_list =[]
        self.blocked_demand_list = []

        # iterate over all demands
        #for demand in self.generated_demands:
        for index in self.generated_demands.index:
            demand = self.generated_demands.iloc[index]


            no_demand = demand['no_demands']
            data_rate = demand['data_rate']
            source = demand['source']
            destination = demand['destination']

            print(f'\t\t\t\t\t ******  Demand  at this level is [{no_demand},{data_rate},({source},{destination})]  ******')

            # calculate input traffic
            self.input_traffic = no_demand*data_rate

            # call channel_attribute to first find short paths between source and destination,
            # then set channel attributes to these paths
            self.channel_attribute(source=source,destination=destination)

            # iterate over all short paths available for the demand
            for shortest_path_name in self.shortest_paths_with_channels.keys():


                # Chose one shortest path
                chosen_shortest_path= self.shortest_paths_with_channels[shortest_path_name] # loop for all shortest_paths
                print('\nFor this demand, chosed shortest path is--->',chosen_shortest_path)



                positive_counter = 0
                req_channles_dict = {}
                # calculate distance

                self.total_distance = 0
                for link in chosen_shortest_path.keys():
                  self.total_distance += self.G[link[0]][link[1]]['weight']

                if self.total_distance <=1300:
                  print(f'Set data_rate to 500, since total_distance <= 1300')
                  self.data_rate=500

                elif 1300 < self.total_distance <= 2500:
                  print(f'Set data_rate to 400, since 1300 < total_distan <= 2500')
                  self.data_rate=400
                else:
                  self.next_shortest_path = True

                # iterate over all links of chosen the shortest path
                for link in chosen_shortest_path.keys():

                      self.free_channel = chosen_shortest_path[link]['free_channels']


                      # add input traffic of this link to input_traffic_list to use in FR, AVO and ...
                      self.input_traffic_list.append(self.input_traffic)

                      # calculate the number WDM trails(required_channels) and it to required_channels_list
                      self.required_channels = math.ceil(self.input_traffic / self.data_rate)
                      self.required_channels_list.append(self.required_channels)
                      req_channles_dict[link] = self.required_channels



                      if self.free_channel>=self.required_channels:
                            positive_counter+=1

                print(f'Required channels(WDMS) for chosed path is:{(self.input_traffic / self.data_rate)}-->{self.required_channels}')

                # If positive_counter == number of links in a chosen short path, it means all links have appropriate
                # capacity to pass the demand
                if positive_counter == len(chosen_shortest_path.keys()):

                    # pass required_channels from each link of chosen path and update free channels
                    for link in chosen_shortest_path.keys():
                            self.required_channels = req_channles_dict[link]
                            self.shortest_paths_with_channels[shortest_path_name][link]['free_channels']-= self.required_channels
                            self.G[link[0]][link[1]]['channels'] -= self.required_channels
                    print('\nUpdataed chosed_shortest_path is: ',self.shortest_paths_with_channels[shortest_path_name])
                    print('='*200)


                    # Here, we use break, since the chosen pass could completely pass the demand and there is no need
                    # to check other short paths for this demand
                    break


                # If positive_counter != number of links in a chosen short path, it means some links do not
                # have appropriate capacity to pass the demand. So, we need to find bottleneck of that path
                # and pass demand == bottleneck from that chosen path and reserve the rest for next chosen_path
                else:

                    # Find bottleneck. First initial bottleneck is equal to 96 * 400
                    bottleneck =96*400

                    # look at each link . If capacity of that link < bottleneck, we consider it as the new_bottleneck
                    for link in chosen_shortest_path.keys():
                            current_bottleneck = self.shortest_paths_with_channels[shortest_path_name][link]['free_channels'] * self.data_rate

                            if current_bottleneck < bottleneck:
                                bottleneck = current_bottleneck

                    # This part remove some input traffic from list in previous
                    for link in chosen_shortest_path.keys() :
                            self.input_traffic_list.pop()
                            self.required_channels_list.pop()

                    # Iterate over each link to see the number of channels we can pass and vice versa.
                    for link in chosen_shortest_path.keys():
                            gbit_able_to_pass = bottleneck
                            gbit_not_able_to_pass = self.input_traffic - bottleneck

                            self.channel_able_to_pass = math.ceil(gbit_able_to_pass / self.data_rate)

                            # For replacing the old input traffic with new bottleneck as new traffic for calculating FR and ...
                            self.input_traffic_list.append(bottleneck)
                            self.required_channels_list.append(self.channel_able_to_pass)
                            if req_channles_dict[link]>= self.shortest_paths_with_channels[shortest_path_name][link]['free_channels']:
                                channel_not_able_to_pass = req_channles_dict[link] - self.channel_able_to_pass
                            else:
                                channel_not_able_to_pass = 0

                            print(f'But for this link({link}) we can pass ({self.channel_able_to_pass}) channels and we must reserve ({channel_not_able_to_pass}) for the next shortest_path')

                            if channel_not_able_to_pass>0:
                                req_channles_dict[link] = channel_not_able_to_pass
                                self.shortest_paths_with_channels[shortest_path_name][link]['free_channels']= 0
                                self.G[link[0]][link[1]]['channels'] = 0

                            elif channel_not_able_to_pass==0:
                                req_channles_dict[link] = 0
                                self.shortest_paths_with_channels[shortest_path_name][link]['free_channels']-= self.channel_able_to_pass
                                self.G[link[0]][link[1]]['channels'] -= self.channel_able_to_pass





                    print('\nPrevious shortest_path has been updated-->',self.shortest_paths_with_channels[shortest_path_name])
                    print('\nBut we have still some required channels. So, we go through another shortest path: ')
                    self.channel_attribute(source=source,destination=destination)# call channel to update free channels
                    self.input_traffic = gbit_not_able_to_pass # For other part


            #Checking block
            # We set minor block which means one shortest path can be block not all shortest paths
            self.minor_block = 0
            for shortest_path_name in self.shortest_paths_with_channels.keys():
                chosen_shortest_path= self.shortest_paths_with_channels[shortest_path_name] # loop for all shortest_paths
                for link in chosen_shortest_path.keys():
                    if chosen_shortest_path[link]['free_channels']==0:
                        self.minor_block +=1
                        break

                # if number of minor block == number of all shortest_paths, means all paths do not have the capacity for
                # passing the demand, and we have blocking problem
            if self.minor_block == len(self.shortest_paths_with_channels.keys()):
                print('Blocking will happen here, and no_of_blocked_demand is: ',no_demand)
                self.blocked_demand_list.append(no_demand)

            else:
                self.blocked_demand_list.append(0)

          ##########################################################################


        # Draw graph for the demand
        #self.draw_graph(title=f'step {self.step}')

        # Calculate all parameters we need for the demand
        self.calculations()

        print('='*200)
        print('50 new random demand added')
        print('='*200)
        # Add another 50 random demands to the demands, avaialble in system
        self.updating()



    ##############################################################################
    def updating(self):
        """ Check Blocking < 1 or not, if so, program will be stopped. Otherwise it addes 50 random demands """

        if self.BR<1:

                self.step +=1
                self.update_demand =True
                self.add_new_demand()
                self.routing()


        else:
          print(f'The network has been stopped because of BlooCking rate {self.BR}')
    def job(self):
      self.AVO = self.FR + 0.05 * self.FR
    ##############################################################################
    def calculations(self):
        """Calculate FR, AVO, BR"""

        self.total_no_of_demands = 0
        self.total_no_of_demands = self.step * 50 + 300
        print(f'So far, total_no_of_demands is :{self.total_no_of_demands}')

        self.no_of_blocked_demand = sum(self.blocked_demand_list)
        self.used_bandwidth = sum(self.input_traffic_list)
        self.used_channels = sum(self.required_channels_list)
        self.FR = (self.used_bandwidth / self.total_bandwidth)*100
        self.AVO = (self.used_channels / self.total_channels)*100
        self.BR = (self.no_of_blocked_demand / self.total_no_of_demands)*100
        #self.job()
        #####
        print(f'used_bandwidth: {self.used_bandwidth } -->{self.input_traffic_list}')
        print(f'used channels: {self.used_channels} --->{self.required_channels_list}')
        print(f'total_channels: {self.total_channels}')
        print('Filling Ratio is: ',self.FR)
        print('Average Fiber Ocupation: ',self.AVO)
        print('Blocking Rate: ',self.BR)
        self.info_dict[f'step {self.step}'] = {'FR':self.FR,'AVO':self.AVO,'BR':self.BR}


    def store_values(self):
        """Return values we need"""

        return self.info_dict, self.generated_demands

In [ ]:
CND = Top_down_continental_with_grooming()
route = CND.routing()

In [14]:
values,generated_demands = CND.store_values()
Top_down_continental_with_grooming_table = pd.DataFrame(values)
Top_down_continental_with_grooming_table.to_csv('Top_down_continental_with_grooming_table.csv')
Top_down_continental_with_grooming_table

,step 0,step 1,step 2,step 3,step 4,step 5,step 6,step 7,step 8,step 9,...,step 12,step 13,step 14,step 15,step 16,step 17,step 18,step 19,step 20,step 21
FR,13.217593,15.162037,17.361111,19.085648,21.446759,23.541667,25.509259,27.870370,29.791667,31.759259,...,38.136574,40.324074,43.136574,45.648148,47.604167,49.409722,51.388889,53.113426,55.208333,55.578704
AVO,15.468750,17.135417,19.375000,20.833333,23.020833,24.895833,26.197917,28.072917,29.895833,31.770833,...,37.864583,39.791667,41.666667,44.166667,46.041667,47.552083,49.427083,50.937500,52.916667,53.541667
BR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.846154,3.037037


# 4: Top_down_continental_without_grooming

In [15]:

class Top_down_continental_without_grooming():

    def __init__(self):
        """Setup initial parameter while creating class"""

        self.update_demand = False
        self.step = 0
        self.info_dict = {}  # to store values

        self.table = pd.read_csv('/content/random_demands.csv')
        self.initial_number = 300
        self.generated_demands = self.table.iloc[:self.initial_number]

        # For Grooming
    def setup_graph(self):

      global distnace_list_cont
      # Create a complete graph with 8 nodes
      self.G = nx.complete_graph(8)

      # List all possible edges
      all_edges = list(self.G.edges())

      # Remove edges until the number of edges reaches
      while len(self.G.edges()) > 20:
          # Remove the last edge from the list
          edge_to_remove = all_edges.pop()

          # Remove the chosen edge
          self.G.remove_edge(*edge_to_remove)

      # Print the edges of the resulting graph
      self.node_list = self.G.nodes()
      self.edge_list = self.G.edges()

      print('links',len(self.edge_list))

      for index, edge in enumerate(self.G.edges()):
        weight = distnace_list_cont[index]
        self.G.add_edge(*edge,weight=weight,channels=96)

      # Fixed variabels
      self.total_bandwidth = 10*500*96 + 10*400*96
      self.total_channels = 20*96
    ##############################################################################
    # def setup_graph(self):
    #     """Setup graph"""

    #     self.G = nx.Graph()
    #     self.G.add_nodes_from([1,2,3,4,5,6])
    #     self.node_color = ['green','blue','red','blue','blue','blue']
    #     self.node_positions = {1: (0, 0), 2: (-.5, 0), 3: (-0.5, 0.5), 4: (0, .5), 5: (.5, .5),6:(.5,0)}
    #     # add edges
    #     self.G.add_edge(1,2,weight=400,channels=96)
    #     self.G.add_edge(1,3,weight=600,channels=96)
    #     self.G.add_edge(1,4,weight=800,channels=96)
    #     self.G.add_edge(1,6,weight=1500,channels=96)
    #     self.G.add_edge(5,6,weight=1600,channels=96)
    #     self.G.add_edge(4,5,weight=400,channels=96)
    #     self.G.add_edge(3,4,weight=800,channels=96)
    #     self.G.add_edge(2,3,weight=900,channels=96)
    #     self.node_list = self.G.nodes()
    #     self.edge_list = self.G.edges()
    #     # Fixed variabels
    #     self.total_bandwidth = 4*500*96 + 4*400*96
    #     self.total_channels = 8*96


    def add_new_demand(self):
        """ad 50 another demands to random_demands table"""

        self.initial_number += 50
        self.generated_demands = self.table.iloc[:self.initial_number]
        #self.grooming()
    ##############################################################################
    # def initial_random_demand(self):
    #     """generate first 300 random demands"""

    #     # Given lists
    #     list1 = self.edge_list
    #     list2 = [100,400]

    #     # Generate all possible combinations
    #     combinations = [(a, b) for a in list2 for b in list1]

    #     # Generate 300 random items
    #     random_items = random.choices(combinations, k=300)

    #     # Count occurrences of each combination
    #     counts = Counter(random_items)

    #     # Merge similar combinations into one list with counts
    #     initial_demand = []
    #     for item, count in counts.items():
    #         if count > 1:
    #             initial_demand.append([count, *item])
    #         else:
    #             initial_demand.append([1,*item])

    #     self.generated_demands = initial_demand

    ##############################################################################
    # def add_new_demand(self):
    #     """Add random demands 50 by 50"""

    #     # Given lists
    #     list1 = self.edge_list
    #     list2 = [100,400]

    #     # Generate all possible combinations
    #     combinations = [(a, b) for a in list2 for b in list1]

    #     # Generate 300 random items
    #     random_items = random.choices(combinations, k=50)

    #     # Count occurrences of each combination
    #     counts = Counter(random_items)

    #     # Merge similar combinations into one list with counts
    #     new_demand = []
    #     for item, count in counts.items():
    #         if count > 1:
    #             new_demand.append([count, *item])
    #         else:
    #             new_demand.append([1,*item])


    #     # add all new_generated demands to initial demands
    #     for demand in new_demand:
    #       self.generated_demands.append(demand)

    ##############################################################################
    def draw_graph(self,title):
        """draw_graph function"""

        nx.draw(self.G,with_labels=True,node_color=self.node_color,pos=self.node_positions)
        self.edge_labels = nx.get_edge_attributes(self.G, 'channels')
        plt.title(title, fontweight="bold")
        nx.draw_networkx_edge_labels(self.G, pos=self.node_positions, edge_labels=self.edge_labels)
        plt.show()

    ##############################################################################
    def shortest_paths(self,source,destination):
        """Find all shortest paths for each source and destination"""

        self.source = source
        self.destination = destination
        paths_dict ={}
        # We use simple paths at the beginning
        all_simple_paths = list(nx.all_simple_paths(self.G, self.source, self.destination)) # l

        index = 1
        for path in all_simple_paths:
          weight = sum(self.G[path[i]][path[i+1]]['weight'] for i in range(len(path)-1))
          paths_dict[f'path_{index}'] = {'path':path,'weight':weight}
          index+=1

        # Sort all simple paths based on their weight to get shortest_paths
        self.sorted_paths = dict(sorted(paths_dict.items(), key=lambda item: item[1]['weight']))

    ##############################################################################
    def channel_attribute(self,source,destination):
        """Add channel attribute to each  link of chosen shortest_path"""

        self.source = source
        self.destination = destination

        # call shortest_path
        self.shortest_paths(self.source,self.destination)
        self.sh_p = self.sorted_paths

        self.shortest_paths_with_channels ={}
        for shortest_path_key in self.sh_p.keys(): # loop for all shortest_paths
            chosen_shortest_path = self.sh_p[shortest_path_key]
            no_links_here =len(chosen_shortest_path['path'])-1

            link_attrs ={}
            for i in range(no_links_here):
              free_channles = self.G[chosen_shortest_path['path'][i]][chosen_shortest_path['path'][i+1]]['channels']
              link_attrs[(chosen_shortest_path['path'][i], chosen_shortest_path['path'][i+1])]={"free_channels":free_channles }


            self.shortest_paths_with_channels[shortest_path_key] = link_attrs

        # print('All shortest paths  with added channels attribute:')


        # for path in self.shortest_paths_with_channels.keys():
        #   print(self.shortest_paths_with_channels[path])

    def grooming(self):

      self.new_table = pd.DataFrame({'no_demands':[],'data_rate':[],'source':[],'destination':[]})
      for source in self.node_list:
        for destination in self.node_list:
          if source != destination:
            for data_rate  in [100,400] :
              result = self.generated_demands[((self.generated_demands.source==source) & (self.generated_demands.destination==destination) & (self.generated_demands.data_rate==data_rate))]
              no_demands = result.shape[0]
              self.new_table.loc[len(self.new_table)] = [no_demands,data_rate,source,destination]

      self.generated_demands = self.new_table
      print('I am in grooming')

    ##############################################################################
    def routing(self):
        """Routing part"""

        # setup_graph
        self.setup_graph()


        # check if there are initial demands in the system or not. If so, we do not need to call initial_random_demand
        if self.update_demand ==False:
          self.initial_number = 300
          self.generated_demands = self.table.iloc[:self.initial_number]
          self.grooming()

        else:
          pass

        self.input_traffic_list = []
        self.required_channels_list =[]
        self.blocked_demand_list = []

        # iterate over all demands
        #for demand in self.generated_demands:
        for index in self.generated_demands.index:
            demand = self.generated_demands.iloc[index]


            no_demand = demand['no_demands']
            data_rate = demand['data_rate']
            source = demand['source']
            destination = demand['destination']

            print(f'\t\t\t\t\t ******  Demand  at this level is [{no_demand},{data_rate},({source},{destination})]  ******')

            # calculate input traffic
            self.input_traffic = no_demand*data_rate

            # call channel_attribute to first find short paths between source and destination,
            # then set channel attributes to these paths
            self.channel_attribute(source=source,destination=destination)

            # iterate over all short paths available for the demand
            for shortest_path_name in self.shortest_paths_with_channels.keys():


                # Chose one shortest path
                chosen_shortest_path= self.shortest_paths_with_channels[shortest_path_name] # loop for all shortest_paths
                print('\nFor this demand, chosed shortest path is--->',chosen_shortest_path)



                positive_counter = 0
                req_channles_dict = {}
                # calculate distance

                self.total_distance = 0
                for link in chosen_shortest_path.keys():
                  self.total_distance += self.G[link[0]][link[1]]['weight']

                if self.total_distance <=1300:
                  print(f'Set data_rate to 500, since total_distance <= 1300')
                  self.data_rate=500

                elif 1300 < self.total_distance <= 2500:
                  print(f'Set data_rate to 400, since 1300 < total_distan <= 2500')
                  self.data_rate=400
                else:
                  self.next_shortest_path = True

                # iterate over all links of chosen the shortest path
                for link in chosen_shortest_path.keys():

                      self.free_channel = chosen_shortest_path[link]['free_channels']


                      # add input traffic of this link to input_traffic_list to use in FR, AVO and ...
                      self.input_traffic_list.append(self.input_traffic)

                      # calculate the number WDM trails(required_channels) and it to required_channels_list
                      self.required_channels = math.ceil(self.input_traffic / self.data_rate)
                      self.required_channels_list.append(self.required_channels)
                      req_channles_dict[link] = self.required_channels



                      if self.free_channel>=self.required_channels:
                            positive_counter+=1

                print(f'Required channels(WDMS) for chosed path is:{(self.input_traffic / self.data_rate)}-->{self.required_channels}')

                # If positive_counter == number of links in a chosen short path, it means all links have appropriate
                # capacity to pass the demand
                if positive_counter == len(chosen_shortest_path.keys()):

                    # pass required_channels from each link of chosen path and update free channels
                    for link in chosen_shortest_path.keys():
                            self.required_channels = req_channles_dict[link]
                            self.shortest_paths_with_channels[shortest_path_name][link]['free_channels']-= self.required_channels
                            self.G[link[0]][link[1]]['channels'] -= self.required_channels
                    print('\nUpdataed chosed_shortest_path is: ',self.shortest_paths_with_channels[shortest_path_name])
                    print('='*200)


                    # Here, we use break, since the chosen pass could completely pass the demand and there is no need
                    # to check other short paths for this demand
                    break


                # If positive_counter != number of links in a chosen short path, it means some links do not
                # have appropriate capacity to pass the demand. So, we need to find bottleneck of that path
                # and pass demand == bottleneck from that chosen path and reserve the rest for next chosen_path
                else:

                    # Find bottleneck. First initial bottleneck is equal to 96 * 400
                    bottleneck =96*400

                    # look at each link . If capacity of that link < bottleneck, we consider it as the new_bottleneck
                    for link in chosen_shortest_path.keys():
                            current_bottleneck = self.shortest_paths_with_channels[shortest_path_name][link]['free_channels'] * self.data_rate

                            if current_bottleneck < bottleneck:
                                bottleneck = current_bottleneck

                    # This part remove some input traffic from list in previous
                    for link in chosen_shortest_path.keys() :
                            self.input_traffic_list.pop()
                            self.required_channels_list.pop()

                    # Iterate over each link to see the number of channels we can pass and vice versa.
                    for link in chosen_shortest_path.keys():
                            gbit_able_to_pass = bottleneck
                            gbit_not_able_to_pass = self.input_traffic - bottleneck

                            self.channel_able_to_pass = math.ceil(gbit_able_to_pass / self.data_rate)

                            # For replacing the old input traffic with new bottleneck as new traffic for calculating FR and ...
                            self.input_traffic_list.append(bottleneck)
                            self.required_channels_list.append(self.channel_able_to_pass)
                            if req_channles_dict[link]>= self.shortest_paths_with_channels[shortest_path_name][link]['free_channels']:
                                channel_not_able_to_pass = req_channles_dict[link] - self.channel_able_to_pass
                            else:
                                channel_not_able_to_pass = 0

                            print(f'But for this link({link}) we can pass ({self.channel_able_to_pass}) channels and we must reserve ({channel_not_able_to_pass}) for the next shortest_path')

                            if channel_not_able_to_pass>0:
                                req_channles_dict[link] = channel_not_able_to_pass
                                self.shortest_paths_with_channels[shortest_path_name][link]['free_channels']= 0
                                self.G[link[0]][link[1]]['channels'] = 0

                            elif channel_not_able_to_pass==0:
                                req_channles_dict[link] = 0
                                self.shortest_paths_with_channels[shortest_path_name][link]['free_channels']-= self.channel_able_to_pass
                                self.G[link[0]][link[1]]['channels'] -= self.channel_able_to_pass





                    print('\nPrevious shortest_path has been updated-->',self.shortest_paths_with_channels[shortest_path_name])
                    print('\nBut we have still some required channels. So, we go through another shortest path: ')
                    self.channel_attribute(source=source,destination=destination)# call channel to update free channels
                    self.input_traffic = gbit_not_able_to_pass # For other part


            #Checking block
            # We set minor block which means one shortest path can be block not all shortest paths
            self.minor_block = 0
            for shortest_path_name in self.shortest_paths_with_channels.keys():
                chosen_shortest_path= self.shortest_paths_with_channels[shortest_path_name] # loop for all shortest_paths
                for link in chosen_shortest_path.keys():
                    if chosen_shortest_path[link]['free_channels']==0:
                        self.minor_block +=1
                        break

                # if number of minor block == number of all shortest_paths, means all paths do not have the capacity for
                # passing the demand, and we have blocking problem
            if self.minor_block == len(self.shortest_paths_with_channels.keys()):
                print('Blocking will happen here, and no_of_blocked_demand is: ',no_demand)
                self.blocked_demand_list.append(no_demand)

            else:
                self.blocked_demand_list.append(0)

          ##########################################################################


        # Draw graph for the demand
        #self.draw_graph(title=f'step {self.step}')

        # Calculate all parameters we need for the demand
        self.calculations()

        print('='*200)
        print('50 new random demand added')
        print('='*200)
        # Add another 50 random demands to the demands, avaialble in system
        self.updating()



    ##############################################################################
    def updating(self):
        """ Check Blocking < 1 or not, if so, program will be stopped. Otherwise it addes 50 random demands """

        if self.BR<1:

                self.step +=1
                self.update_demand =True
                self.add_new_demand()
                self.routing()


        else:
          print(f'The network has been stopped because of BlooCking rate {self.BR}')
    def job(self):
      self.AVO = self.FR + 0.05 * self.FR
    ##############################################################################
    def calculations(self):
        """Calculate FR, AVO, BR"""

        self.total_no_of_demands = 0
        self.total_no_of_demands = self.step * 50 + 300
        print(f'So far, total_no_of_demands is :{self.total_no_of_demands}')

        self.no_of_blocked_demand = sum(self.blocked_demand_list)
        self.used_bandwidth = sum(self.input_traffic_list)
        self.used_channels = sum(self.required_channels_list)
        self.FR = (self.used_bandwidth / self.total_bandwidth)*100
        self.AVO = (self.used_channels / self.total_channels)*100
        self.BR = (self.no_of_blocked_demand / self.total_no_of_demands)*100
        #self.job()
        #####
        print(f'used_bandwidth: {self.used_bandwidth } -->{self.input_traffic_list}')
        print(f'used channels: {self.used_channels} --->{self.required_channels_list}')
        print(f'total_channels: {self.total_channels}')
        print('Filling Ratio is: ',self.FR)
        print('Average Fiber Ocupation: ',self.AVO)
        print('Blocking Rate: ',self.BR)
        self.info_dict[f'step {self.step}'] = {'FR':self.FR,'AVO':self.AVO,'BR':self.BR}


    def store_values(self):
        """Return values we need"""

        return self.info_dict, self.generated_demands

In [ ]:
CND = Top_down_continental_without_grooming()
route = CND.routing()

In [17]:
values,generated_demands = CND.store_values()
Top_down_continental_without_grooming_table = pd.DataFrame(values)
Top_down_continental_without_grooming_table.to_csv('Top_down_continental_without_grooming_table.csv')
Top_down_continental_without_grooming_table

,step 0,step 1,step 2,step 3,step 4,step 5,step 6,step 7,step 8,step 9,step 10,step 11,step 12,step 13
FR,13.217593,15.104167,17.256944,19.108796,21.678241,23.888889,25.902778,28.229167,30.208333,32.430556,34.768519,36.967593,38.958333,40.416667
AVO,15.468750,27.031250,30.625000,34.583333,38.645833,42.500000,46.562500,50.781250,54.687500,58.437500,62.864583,66.979167,70.468750,73.437500
BR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,1.684211


# 5: Bottom_up_national_with_grooming

In [18]:
class Bottom_up_national_with_grooming():


      def __init__(self):
        self.full_mesh()
        self.update_demand = False
        self.step = 0
        self.info_dict = {}  # to store values

        self.table = pd.read_csv('/content/random_demands.csv')
        self.initial_number = 300
        self.generated_demands = self.table.iloc[:self.initial_number]

      def setup_graph(self):

        global distnace_list
        # Create a complete graph with 8 nodes
        self.G = nx.complete_graph(8)

        # List all possible edges
        all_edges = list(self.G.edges())

        # Remove edges until the number of edges reaches
        while len(self.G.edges()) > 16:
            # Remove the last edge from the list
            edge_to_remove = all_edges.pop()

            # Remove the chosen edge
            self.G.remove_edge(*edge_to_remove)

        # Print the edges of the resulting graph
        self.node_list = self.G.nodes()
        self.edge_list = self.G.edges()

        print('links',len(self.edge_list))

        for index, edge in enumerate(self.G.edges()):
          weight = distnace_list[index]
          self.G.add_edge(*edge,weight=weight,channels={'WDM_500':[0],'WDM_400':[0]})


      def full_mesh(self):
        self.paths = []
        self.setup_graph()
        n = 1
        while n<=100:
          for source in self.node_list:
            for destination in self.node_list:
              if source != destination:
                self.path = (source,destination)
                self.paths.append(self.path)
          n +=1
        # repated this full mesh for 100 times

      def add_new_demand(self):
          """ad 50 another demands to random_demands table"""

          self.initial_number += 50
          self.generated_demands = self.table.iloc[:self.initial_number]
          self.grooming()




      def channel_attribute(self,source,destination):
          """Add channel attribute to each  link of chosen shortest_path"""

          self.source = source
          self.destination = destination

          # call shortest_path
          self.shortest_paths(self.source,self.destination)
          self.sh_p = self.sorted_paths

          self.shortest_paths_with_channels ={}
          for shortest_path_key in self.sh_p.keys(): # loop for all shortest_paths
              chosen_shortest_path = self.sh_p[shortest_path_key]
              no_links_here =len(chosen_shortest_path['path'])-1

              link_attrs ={}
              for i in range(no_links_here):
                channels = self.G[chosen_shortest_path['path'][i]][chosen_shortest_path['path'][i+1]]['channels']
                link_attrs[(chosen_shortest_path['path'][i], chosen_shortest_path['path'][i+1])]={"setup_channels":channels }


              self.shortest_paths_with_channels[shortest_path_key] = link_attrs

################################################################################
      def grooming(self):

          self.new_table = pd.DataFrame({'no_demands':[],'data_rate':[],'source':[],'destination':[]})
          for source in self.node_list:
            for destination in self.node_list:
              if source != destination:
                for data_rate  in [100,400] :
                  result = self.generated_demands[((self.generated_demands.source==source) & (self.generated_demands.destination==destination) & (self.generated_demands.data_rate==data_rate))]
                  no_demands = result.shape[0]
                  self.new_table.loc[len(self.new_table)] = [no_demands,data_rate,source,destination]

          self.generated_demands = self.new_table
          print('I am in grooming')
################################################################################
      def set_wdm(self):
          for path in self.paths:
              #print(f'path is like this: {path}')
              source = path[0]
              destination = path[1]
              # then set channel attributes to these paths
              self.channel_attribute(source=source, destination=destination)

              counter = 0
              for shortest_path_name in self.shortest_paths_with_channels.keys():  # loop for all shortest_paths
                  chosen_shortest_path = self.shortest_paths_with_channels[shortest_path_name]
                  #print('\nFor this path, chosed shortest path is--->', chosen_shortest_path)

                  positive_counter = 0
                  req_channles_dict = {}

                  for link in chosen_shortest_path.keys():

                      number_of_wdms_500 = chosen_shortest_path[link]['setup_channels']['WDM_500'][0]
                      number_of_wdms_400 = chosen_shortest_path[link]['setup_channels']['WDM_400'][0]
                      number_of_wdms = number_of_wdms_500 + number_of_wdms_400

                      self.setup_channels = number_of_wdms

                      if self.setup_channels < 96:

                          #print(f'yes {self.setup_channels}({number_of_wdms_500} + {number_of_wdms_400}) <96')
                          positive_counter += 1

                  #print(f'positive_counter : {positive_counter}')

                  self.total_distance = 0
                  for link in chosen_shortest_path.keys():
                      self.total_distance += self.G[link[0]][link[1]]['weight']

                  if self.total_distance <= 1300:
                      #print(f'we must set 1 WDM with 500 Gb/s since distance <=1300')
                      self.WDM_data_rate = 500

                  elif 1300 < self.total_distance <= 2500:
                      #print(f'we must set 1 WDM with 400 Gb/s since 1300 <distance <=2500')
                      self.WDM_data_rate = 400
                  else:
                      self.next_shortest_path = True

                  # capacity to pass the demand
                  if positive_counter == len(chosen_shortest_path.keys()):

                      # pass required_channels from each link of chosen path and update free channels
                      for link in chosen_shortest_path.keys():
                          if self.WDM_data_rate == 500:
                            old_wdm_500 = self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0]
                            self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0] = old_wdm_500 + 1
                          else:
                            old_wdm_400 = self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0]
                            self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0] = old_wdm_400 + 1
                          # self.G[link[0]][link[1]]['channels'].append(self.WDM_data_rate)

                      #print('\nUpdataed chosed_shortest_path is: ', self.shortest_paths_with_channels[shortest_path_name])
                      #print('=' * 200)

                      # Here, we use break, since the chosen pass could completely pass the demand and there is no need
                      # to check other short paths for this demand
                      break
                  counter += 1

              if counter == len(self.shortest_paths_with_channels.keys()):
                  #print('there is no space to setup wdm')
                  #self.draw_graph()
                  self.calculation1()
                  break





################################################################################
      def draw_graph(self,title='now'):
          """draw_graph function"""

          nx.draw(self.G,with_labels=True,node_color=self.node_color,pos=self.node_positions)
          self.edge_labels = nx.get_edge_attributes(self.G, 'channels')
          plt.title(title, fontweight="bold")
          nx.draw_networkx_edge_labels(self.G, pos=self.node_positions, edge_labels=self.edge_labels)
          plt.show()
          print(self.edge_labels)

################################################################################
      def shortest_paths(self,source,destination):
          """Find all shortest paths for each source and destination"""

          self.source = source
          self.destination = destination
          paths_dict ={}
          # We use simple paths at the beginning
          all_simple_paths = list(nx.all_simple_paths(self.G, self.source, self.destination)) # l

          index = 1
          for path in all_simple_paths:
            weight = sum(self.G[path[i]][path[i+1]]['weight'] for i in range(len(path)-1))
            paths_dict[f'path_{index}'] = {'path':path,'weight':weight,'setup_channels':self.G[path[0]][path[1]]['channels'] }
            index+=1

          # Sort all simple paths based on their weight to get shortest_paths
          self.sorted_paths = dict(sorted(paths_dict.items(), key=lambda item: item[1]['weight']))

################################################################################
      def routing(self):

        # set graph and wdm
        self.set_wdm()


        # check if there are initial demands in the system or not. If so, we do not need to call initial_random_demand
        if self.update_demand ==False:
          self.initial_number = 300
          self.generated_demands = self.table.iloc[:self.initial_number]
          self.grooming()

        else:
          pass


        self.input_traffic_list = []
        self.required_channels_list =[]
        self.blocked_demand_list = []


        # iterate over all demands
        #for demand in self.generated_demands:
        jj = 0
        for index in self.generated_demands.index:
            demand = self.generated_demands.iloc[index]


            no_demand = demand['no_demands']
            data_rate = demand['data_rate']
            source = demand['source']
            destination = demand['destination']
            print('*'*100)
            print(f'\t\t\t\t\t ******  Demand {index} at this level is [{no_demand},{data_rate},({source},{destination})]  ******')

            # calculate input traffic
            self.input_traffic = no_demand*data_rate

            # call channel_attribute to first find short paths between source and destination,
            # then set channel attributes to these paths
            self.channel_attribute(source=source,destination=destination)


            # iterate over all short paths available for the demand
            for shortest_path_name in self.shortest_paths_with_channels.keys():

                # Chose one shortest path
                chosen_shortest_path= self.shortest_paths_with_channels[shortest_path_name] # loop for all shortest_paths
                print('\nFor this demand, chosed shortest path is--->',chosen_shortest_path)

                positive_counter_500 = 0
                positive_counter_400 = 0
                req_channles_dict = {}


                self.total_distance = 0
                for link in chosen_shortest_path.keys():
                  self.total_distance += self.G[link[0]][link[1]]['weight']


                if self.total_distance <=1300:
                  print(f'Set data_rate to 500, since total_distance <= 1300')
                  self.data_rate=500

                elif 1300 < self.total_distance <= 2500:
                  print(f'Set data_rate to 400, since 1300 < total_distan <= 2500')
                  self.data_rate=400
                else:
                  self.next_shortest_path = True


                # iterate over all links of chosen the shortest path
                for link in chosen_shortest_path.keys():

                      self.free_channel_500 = chosen_shortest_path[link]['setup_channels']['WDM_500'][0]
                      self.free_channel_400 = chosen_shortest_path[link]['setup_channels']['WDM_400'][0]


                      # add input traffic of this link to input_traffic_list to use in FR, AVO and ...
                      self.input_traffic_list.append(self.input_traffic)

                      # calculate the number WDM trails(required_channels) and add it to required_channels_list
                      self.required_channels = math.ceil(self.input_traffic / self.data_rate)

                      self.required_channels_list.append(self.required_channels)
                      req_channles_dict[link] = self.required_channels


                      if self.data_rate == 500  and self.free_channel_500 >= self.required_channels:
                            positive_counter_500 +=1

                      elif self.data_rate == 400  and self.free_channel_400 >= self.required_channels:
                            positive_counter_400 +=1


                print(f'Required channels(WDMS) for chosed path is:{(self.input_traffic / self.data_rate)}-->{self.required_channels}')



                # If positive_counter == number of links in a chosen short path, it means all links have appropriate
                # capacity to pass the demand
                if positive_counter_500 == len(chosen_shortest_path.keys()):


                    # pass required_channels from each link of chosen path and update free channels
                    for link in chosen_shortest_path.keys():
                            self.required_channels = req_channles_dict[link]
                            self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0]-= self.required_channels
                            #self.G[link[0]][link[1]]['channels']['WDM_500'][0] -= self.required_channels
                    print('\nUpdataed chosed_shortest_path is: ',self.shortest_paths_with_channels[shortest_path_name])
                    print('='*200)


                    # Here, we use break, since the chosen pass could completely pass the demand and there is no need
                    # to check other short paths for this demand
                    break

                elif positive_counter_400 == len(chosen_shortest_path.keys()):


                    # pass required_channels from each link of chosen path and update free channels
                    for link in chosen_shortest_path.keys():
                            self.required_channels = req_channles_dict[link]
                            self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0]-= self.required_channels
                            #self.G[link[0]][link[1]]['channels']['WDM_400'][0] -= self.required_channels
                    print('\nUpdataed chosed_shortest_path is: ',self.shortest_paths_with_channels[shortest_path_name])



                    # Here, we use break, since the chosen pass could completely pass the demand and there is no need
                    # to check other short paths for this demand
                    break


                # If positive_counter_500 or 400 != number of links in a chosen short path, it means some links do not
                # have appropriate capacity to pass the demand. So, we need to find bottleneck of that path
                # and pass demand == bottleneck from that chosen path and reserve the rest for next chosen_path
                else:

                  if  self.data_rate ==400:
                    # Find bottleneck. First initial bottleneck
                    bottleneck = 96*400

                    # look at each link . If capacity of that link < bottleneck, we consider it as the new_bottleneck
                    for link in chosen_shortest_path.keys():
                            current_bottleneck = self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0] * self.data_rate

                            if current_bottleneck < bottleneck:
                                bottleneck = current_bottleneck
                  else:
                    bottleneck = 96*500

                    # look at each link . If capacity of that link < bottleneck, we consider it as the new_bottleneck
                    for link in chosen_shortest_path.keys():
                            current_bottleneck = self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0] * self.data_rate

                            if current_bottleneck < bottleneck:
                                bottleneck = current_bottleneck


                  # This part remove some input traffic from list in previous
                  for link in chosen_shortest_path.keys() :
                          self.input_traffic_list.pop()
                          self.required_channels_list.pop()




                  # Iterate over each link to see the number of channels we can pass and vice versa.
                  for link in chosen_shortest_path.keys():
                          gbit_able_to_pass = bottleneck
                          gbit_not_able_to_pass = self.input_traffic - bottleneck

                          self.channel_able_to_pass = math.ceil(gbit_able_to_pass / self.data_rate)

                          # For replacing the old input traffic with new bottleneck as new traffic for calculating FR and ...
                          self.input_traffic_list.append(bottleneck)
                          self.required_channels_list.append(self.channel_able_to_pass)

                          if self.data_rate ==500:
                            if req_channles_dict[link]>= self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0]:
                                channel_not_able_to_pass = req_channles_dict[link] - self.channel_able_to_pass
                            else:
                                channel_not_able_to_pass = 0
                          else:
                            if req_channles_dict[link]>= self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0]:
                                channel_not_able_to_pass = req_channles_dict[link] - self.channel_able_to_pass
                            else:
                                channel_not_able_to_pass = 0

                          print(f'But for this link({link}) we can pass ({self.channel_able_to_pass}) channels with {self.data_rate}Gb/s and we must reserve ({channel_not_able_to_pass}) for the next shortest_path')


                          if self.data_rate ==500:
                            if channel_not_able_to_pass>0:
                                req_channles_dict[link] = channel_not_able_to_pass
                                self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0]= 0
                                self.G[link[0]][link[1]]['channels']['WDM_500'][0] = 0

                            elif channel_not_able_to_pass==0:
                                req_channles_dict[link] = 0
                                self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0]-= self.channel_able_to_pass
                                self.G[link[0]][link[1]]['channels']['WDM_500'][0] -= self.channel_able_to_pass

                          else:

                            if channel_not_able_to_pass>0:
                                req_channles_dict[link] = channel_not_able_to_pass
                                self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0]= 0
                                self.G[link[0]][link[1]]['channels']['WDM_400'][0] = 0

                            elif channel_not_able_to_pass==0:
                                req_channles_dict[link] = 0
                                self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0]-= self.channel_able_to_pass
                                self.G[link[0]][link[1]]['channels']['WDM_400'][0] -= self.channel_able_to_pass

                  print('\nPrevious shortest_path has been updated-->',self.shortest_paths_with_channels[shortest_path_name])
                  print('\nBut we have still some required channels. So, we go through another shortest path: ')
                  self.channel_attribute(source=source,destination=destination)# call channel to update free channels
                  self.input_traffic = gbit_not_able_to_pass # For other part


            #Checking block
            # We set minor block which means one shortest path can be block not all shortest paths
            self.minor_block_500 = 0
            self.minor_block_400 = 0
            for shortest_path_name in self.shortest_paths_with_channels.keys():
                chosen_shortest_path= self.shortest_paths_with_channels[shortest_path_name] # loop for all shortest_paths

                self.total_distance1 = 0
                for link in chosen_shortest_path.keys():
                  self.total_distance1 += self.G[link[0]][link[1]]['weight']

                if self.total_distance1 <= 1300:
                  for link in chosen_shortest_path.keys():

                      if chosen_shortest_path[link]['setup_channels']['WDM_500'][0]==0:
                          self.minor_block_500 +=1
                          break


                elif 1300 < self.total_distance1 :
                  for link in chosen_shortest_path.keys():

                      if chosen_shortest_path[link]['setup_channels']['WDM_400'][0]==0:
                          self.minor_block_400 +=1
                          break


            if (self.minor_block_400 + self.minor_block_500) == len(self.shortest_paths_with_channels.keys()):
                print('Blocking will happen here, and no_of_blocked_demand for this demand is: ',no_demand)
                print('self.minor_block_400',self.minor_block_400)
                print('self.minor_bloc_500',self.minor_block_500)
                self.blocked_demand_list.append(no_demand)

            else:
                print('number of shortest_paths:',len(self.shortest_paths_with_channels.keys()))
                print('self.minor_block_400:',self.minor_block_400)
                print('self.minor_block_500:',self.minor_block_500)
                self.blocked_demand_list.append(0)


        # Calculate all parameters we need for the demand
            self.calculations()


        print('='*200)
        print('50 new random demand added')
        print('='*200)
        # Add another 50 random demands to the demands, avaialble in system
        # if no_continue == False:
        self.updating()


################################################################################
      def updating(self):
          """ Check Blocking < 1 or not, if so, program will be stopped. Otherwise it addes 50 random demands """

          if self.BR<1:

                  self.step +=1
                  self.update_demand =True
                  self.add_new_demand()
                  self.routing()


          else:
            print(f'The network has been stopped because of BlooCking rate {self.BR}')

################################################################################
      def calculations(self):
          """Calculate FR, AVO, BR"""

          self.total_no_of_demands = 0
          self.total_no_of_demands = self.step * 50 + 300
          print(f'So far, total_no_of_demands is :{self.total_no_of_demands}')

          self.no_of_blocked_demand = sum(self.blocked_demand_list)
          self.used_bandwidth = sum(self.input_traffic_list)
          self.used_channels = sum(self.required_channels_list)


          #self.calculation1()
          self.FR = (self.used_bandwidth / self.total_bandwidth)*100
          self.AVO = (self.used_channels / self.total_channels)*100
          self.BR = (self.no_of_blocked_demand / self.total_no_of_demands)*100
          #####
          print('total_no_blocked_demands',self.no_of_blocked_demand)
          print('self.total_bandwidth',self.total_bandwidth)
          print('self.total_channels',self.total_channels)
          print('FR = (self.used_bandwidth / self.total_bandwidth)*100')
          print('AVO = (self.used_channels / self.total_channels)*100')
          print(f'used_bandwidth: {self.used_bandwidth } -->{self.input_traffic_list}')
          print(f'used channels: {self.used_channels} --->{self.required_channels_list}')
          print(f'total_channels: {self.total_channels}')
          print('Filling Ratio is: ',self.FR)
          print('Average Fiber Ocupation: ',self.AVO)
          print('Blocking Rate: ',self.BR)
          self.info_dict[f'step {self.step}'] = {'FR':self.FR,'AVO':self.AVO,'BR':self.BR}

################################################################################
      def calculation1(self):
        self.edge_labels = nx.get_edge_attributes(self.G, 'channels')
        no_of_wdm_500 = 0
        no_of_wdm_400 = 0
        for link in self.edge_labels.keys():
          no_of_wdm_500 += self.edge_labels[link]['WDM_500'][0]
          no_of_wdm_400 += self.edge_labels[link]['WDM_400'][0]

        self.total_bandwidth = no_of_wdm_500 *500 + no_of_wdm_400 *400
        self.total_channels = no_of_wdm_500 + no_of_wdm_400

      def store_values(self):
        """Return values we need"""

        return self.info_dict, self.generated_demands





In [19]:
CND = Bottom_up_national_with_grooming()
route = CND.routing()

Streaming output truncated to the last 5000 lines.
For this demand, chosed shortest path is---> {(7, 0): {'setup_channels': {'WDM_500': [0], 'WDM_400': [0]}}, (0, 4): {'setup_channels': {'WDM_500': [37], 'WDM_400': [0]}}}
Set data_rate to 500, since total_distance <= 1300
Required channels(WDMS) for chosed path is:1.6-->2
But for this link((7, 0)) we can pass (0) channels with 500Gb/s and we must reserve (2) for the next shortest_path
But for this link((0, 4)) we can pass (0) channels with 500Gb/s and we must reserve (0) for the next shortest_path

Previous shortest_path has been updated--> {(7, 0): {'setup_channels': {'WDM_500': [0], 'WDM_400': [0]}}, (0, 4): {'setup_channels': {'WDM_500': [37], 'WDM_400': [0]}}}

But we have still some required channels. So, we go through another shortest path: 

For this demand, chosed shortest path is---> {(7, 0): {'setup_channels': {'WDM_500': [0], 'WDM_400': [0]}}, (0, 1): {'setup_channels': {'WDM_500': [33], 'WDM_400': [0]}}, (1, 4): {'setup_cha

In [20]:
values,generated_demands = CND.store_values()
Bottom_up_national_with_grooming_table = pd.DataFrame(values)
Bottom_up_national_with_grooming_table.to_csv('Bottom_up_national_with_grooming.csv')
Bottom_up_national_with_grooming_table

,step 0,step 1,step 2,step 3,step 4,step 5,step 6,step 7,step 8,step 9,step 10,step 11,step 12,step 13,step 14,step 15
FR,19.696243,21.795472,24.925723,26.585920,29.260355,31.341552,33.523538,36.053748,38.166550,40.568655,42.472527,43.900135,48.907446,52.560455,59.034175,61.931034
AVO,25.579536,27.478532,30.961689,32.172597,34.837278,36.765772,38.231098,40.452617,42.477257,45.076283,47.252747,48.650472,54.140571,57.752489,63.595840,66.590038
BR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.380952


# 6: Bottom_up_national_without_grooming

In [21]:
class Bottom_up_national_without_grooming():


      def __init__(self):
        self.full_mesh()
        self.update_demand = False
        self.step = 0
        self.info_dict = {}  # to store values

        self.table = pd.read_csv('/content/random_demands.csv')
        self.initial_number = 300
        self.generated_demands = self.table.iloc[:self.initial_number]

      def setup_graph(self):

        global distnace_list
        # Create a complete graph with 8 nodes
        self.G = nx.complete_graph(8)

        # List all possible edges
        all_edges = list(self.G.edges())

        # Remove edges until the number of edges reaches
        while len(self.G.edges()) > 16:
            # Remove the last edge from the list
            edge_to_remove = all_edges.pop()

            # Remove the chosen edge
            self.G.remove_edge(*edge_to_remove)

        # Print the edges of the resulting graph
        self.node_list = self.G.nodes()
        self.edge_list = self.G.edges()

        print('links',len(self.edge_list))

        for index, edge in enumerate(self.G.edges()):
          weight = distnace_list[index]
          self.G.add_edge(*edge,weight=weight,channels={'WDM_500':[0],'WDM_400':[0]})


      def full_mesh(self):
        self.paths = []
        self.setup_graph()
        n = 1
        while n<=100:
          for source in self.node_list:
            for destination in self.node_list:
              if source != destination:
                self.path = (source,destination)
                self.paths.append(self.path)
          n +=1
        # repated this full mesh for 100 times

      def add_new_demand(self):
          """ad 50 another demands to random_demands table"""

          self.initial_number += 50
          self.generated_demands = self.table.iloc[:self.initial_number]
          #self.grooming()




      def channel_attribute(self,source,destination):
          """Add channel attribute to each  link of chosen shortest_path"""

          self.source = source
          self.destination = destination

          # call shortest_path
          self.shortest_paths(self.source,self.destination)
          self.sh_p = self.sorted_paths

          self.shortest_paths_with_channels ={}
          for shortest_path_key in self.sh_p.keys(): # loop for all shortest_paths
              chosen_shortest_path = self.sh_p[shortest_path_key]
              no_links_here =len(chosen_shortest_path['path'])-1

              link_attrs ={}
              for i in range(no_links_here):
                channels = self.G[chosen_shortest_path['path'][i]][chosen_shortest_path['path'][i+1]]['channels']
                link_attrs[(chosen_shortest_path['path'][i], chosen_shortest_path['path'][i+1])]={"setup_channels":channels }


              self.shortest_paths_with_channels[shortest_path_key] = link_attrs

################################################################################
      def grooming(self):

          self.new_table = pd.DataFrame({'no_demands':[],'data_rate':[],'source':[],'destination':[]})
          for source in self.node_list:
            for destination in self.node_list:
              if source != destination:
                for data_rate  in [100,400] :
                  result = self.generated_demands[((self.generated_demands.source==source) & (self.generated_demands.destination==destination) & (self.generated_demands.data_rate==data_rate))]
                  no_demands = result.shape[0]
                  self.new_table.loc[len(self.new_table)] = [no_demands,data_rate,source,destination]

          self.generated_demands = self.new_table
          print('I am in grooming')
################################################################################
      def set_wdm(self):
          for path in self.paths:
              #print(f'path is like this: {path}')
              source = path[0]
              destination = path[1]
              # then set channel attributes to these paths
              self.channel_attribute(source=source, destination=destination)

              counter = 0
              for shortest_path_name in self.shortest_paths_with_channels.keys():  # loop for all shortest_paths
                  chosen_shortest_path = self.shortest_paths_with_channels[shortest_path_name]
                  #print('\nFor this path, chosed shortest path is--->', chosen_shortest_path)

                  positive_counter = 0
                  req_channles_dict = {}

                  for link in chosen_shortest_path.keys():

                      number_of_wdms_500 = chosen_shortest_path[link]['setup_channels']['WDM_500'][0]
                      number_of_wdms_400 = chosen_shortest_path[link]['setup_channels']['WDM_400'][0]
                      number_of_wdms = number_of_wdms_500 + number_of_wdms_400

                      self.setup_channels = number_of_wdms

                      if self.setup_channels < 96:

                          #print(f'yes {self.setup_channels}({number_of_wdms_500} + {number_of_wdms_400}) <96')
                          positive_counter += 1

                  #print(f'positive_counter : {positive_counter}')

                  self.total_distance = 0
                  for link in chosen_shortest_path.keys():
                      self.total_distance += self.G[link[0]][link[1]]['weight']

                  if self.total_distance <= 1300:
                      #print(f'we must set 1 WDM with 500 Gb/s since distance <=1300')
                      self.WDM_data_rate = 500

                  elif 1300 < self.total_distance <= 2500:
                      #print(f'we must set 1 WDM with 400 Gb/s since 1300 <distance <=2500')
                      self.WDM_data_rate = 400
                  else:
                      self.next_shortest_path = True

                  # capacity to pass the demand
                  if positive_counter == len(chosen_shortest_path.keys()):

                      # pass required_channels from each link of chosen path and update free channels
                      for link in chosen_shortest_path.keys():
                          if self.WDM_data_rate == 500:
                            old_wdm_500 = self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0]
                            self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0] = old_wdm_500 + 1
                          else:
                            old_wdm_400 = self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0]
                            self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0] = old_wdm_400 + 1
                          # self.G[link[0]][link[1]]['channels'].append(self.WDM_data_rate)

                      #print('\nUpdataed chosed_shortest_path is: ', self.shortest_paths_with_channels[shortest_path_name])
                      #print('=' * 200)

                      # Here, we use break, since the chosen pass could completely pass the demand and there is no need
                      # to check other short paths for this demand
                      break
                  counter += 1

              if counter == len(self.shortest_paths_with_channels.keys()):
                  #print('there is no space to setup wdm')
                  #self.draw_graph()
                  self.calculation1()
                  break





################################################################################
      def draw_graph(self,title='now'):
          """draw_graph function"""

          nx.draw(self.G,with_labels=True,node_color=self.node_color,pos=self.node_positions)
          self.edge_labels = nx.get_edge_attributes(self.G, 'channels')
          plt.title(title, fontweight="bold")
          nx.draw_networkx_edge_labels(self.G, pos=self.node_positions, edge_labels=self.edge_labels)
          plt.show()
          print(self.edge_labels)

################################################################################
      def shortest_paths(self,source,destination):
          """Find all shortest paths for each source and destination"""

          self.source = source
          self.destination = destination
          paths_dict ={}
          # We use simple paths at the beginning
          all_simple_paths = list(nx.all_simple_paths(self.G, self.source, self.destination)) # l

          index = 1
          for path in all_simple_paths:
            weight = sum(self.G[path[i]][path[i+1]]['weight'] for i in range(len(path)-1))
            paths_dict[f'path_{index}'] = {'path':path,'weight':weight,'setup_channels':self.G[path[0]][path[1]]['channels'] }
            index+=1

          # Sort all simple paths based on their weight to get shortest_paths
          self.sorted_paths = dict(sorted(paths_dict.items(), key=lambda item: item[1]['weight']))

################################################################################
      def routing(self):

        # set graph and wdm
        self.set_wdm()


        # check if there are initial demands in the system or not. If so, we do not need to call initial_random_demand
        if self.update_demand ==False:
          self.initial_number = 300
          self.generated_demands = self.table.iloc[:self.initial_number]
          self.grooming()

        else:
          pass


        self.input_traffic_list = []
        self.required_channels_list =[]
        self.blocked_demand_list = []


        # iterate over all demands
        #for demand in self.generated_demands:
        jj = 0
        for index in self.generated_demands.index:
            demand = self.generated_demands.iloc[index]


            no_demand = demand['no_demands']
            data_rate = demand['data_rate']
            source = demand['source']
            destination = demand['destination']
            print('*'*100)
            print(f'\t\t\t\t\t ******  Demand {index} at this level is [{no_demand},{data_rate},({source},{destination})]  ******')

            # calculate input traffic
            self.input_traffic = no_demand*data_rate

            # call channel_attribute to first find short paths between source and destination,
            # then set channel attributes to these paths
            self.channel_attribute(source=source,destination=destination)


            # iterate over all short paths available for the demand
            for shortest_path_name in self.shortest_paths_with_channels.keys():

                # Chose one shortest path
                chosen_shortest_path= self.shortest_paths_with_channels[shortest_path_name] # loop for all shortest_paths
                print('\nFor this demand, chosed shortest path is--->',chosen_shortest_path)

                positive_counter_500 = 0
                positive_counter_400 = 0
                req_channles_dict = {}


                self.total_distance = 0
                for link in chosen_shortest_path.keys():
                  self.total_distance += self.G[link[0]][link[1]]['weight']


                if self.total_distance <=1300:
                  print(f'Set data_rate to 500, since total_distance <= 1300')
                  self.data_rate=500

                elif 1300 < self.total_distance <= 2500:
                  print(f'Set data_rate to 400, since 1300 < total_distan <= 2500')
                  self.data_rate=400
                else:
                  self.next_shortest_path = True


                # iterate over all links of chosen the shortest path
                for link in chosen_shortest_path.keys():

                      self.free_channel_500 = chosen_shortest_path[link]['setup_channels']['WDM_500'][0]
                      self.free_channel_400 = chosen_shortest_path[link]['setup_channels']['WDM_400'][0]


                      # add input traffic of this link to input_traffic_list to use in FR, AVO and ...
                      self.input_traffic_list.append(self.input_traffic)

                      # calculate the number WDM trails(required_channels) and add it to required_channels_list
                      self.required_channels = math.ceil(self.input_traffic / self.data_rate)

                      self.required_channels_list.append(self.required_channels)
                      req_channles_dict[link] = self.required_channels


                      if self.data_rate == 500  and self.free_channel_500 >= self.required_channels:
                            positive_counter_500 +=1

                      elif self.data_rate == 400  and self.free_channel_400 >= self.required_channels:
                            positive_counter_400 +=1


                print(f'Required channels(WDMS) for chosed path is:{(self.input_traffic / self.data_rate)}-->{self.required_channels}')



                # If positive_counter == number of links in a chosen short path, it means all links have appropriate
                # capacity to pass the demand
                if positive_counter_500 == len(chosen_shortest_path.keys()):


                    # pass required_channels from each link of chosen path and update free channels
                    for link in chosen_shortest_path.keys():
                            self.required_channels = req_channles_dict[link]
                            self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0]-= self.required_channels
                            #self.G[link[0]][link[1]]['channels']['WDM_500'][0] -= self.required_channels
                    print('\nUpdataed chosed_shortest_path is: ',self.shortest_paths_with_channels[shortest_path_name])
                    print('='*200)


                    # Here, we use break, since the chosen pass could completely pass the demand and there is no need
                    # to check other short paths for this demand
                    break

                elif positive_counter_400 == len(chosen_shortest_path.keys()):


                    # pass required_channels from each link of chosen path and update free channels
                    for link in chosen_shortest_path.keys():
                            self.required_channels = req_channles_dict[link]
                            self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0]-= self.required_channels
                            #self.G[link[0]][link[1]]['channels']['WDM_400'][0] -= self.required_channels
                    print('\nUpdataed chosed_shortest_path is: ',self.shortest_paths_with_channels[shortest_path_name])



                    # Here, we use break, since the chosen pass could completely pass the demand and there is no need
                    # to check other short paths for this demand
                    break


                # If positive_counter_500 or 400 != number of links in a chosen short path, it means some links do not
                # have appropriate capacity to pass the demand. So, we need to find bottleneck of that path
                # and pass demand == bottleneck from that chosen path and reserve the rest for next chosen_path
                else:

                  if  self.data_rate ==400:
                    # Find bottleneck. First initial bottleneck
                    bottleneck = 96*400

                    # look at each link . If capacity of that link < bottleneck, we consider it as the new_bottleneck
                    for link in chosen_shortest_path.keys():
                            current_bottleneck = self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0] * self.data_rate

                            if current_bottleneck < bottleneck:
                                bottleneck = current_bottleneck
                  else:
                    bottleneck = 96*500

                    # look at each link . If capacity of that link < bottleneck, we consider it as the new_bottleneck
                    for link in chosen_shortest_path.keys():
                            current_bottleneck = self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0] * self.data_rate

                            if current_bottleneck < bottleneck:
                                bottleneck = current_bottleneck


                  # This part remove some input traffic from list in previous
                  for link in chosen_shortest_path.keys() :
                          self.input_traffic_list.pop()
                          self.required_channels_list.pop()




                  # Iterate over each link to see the number of channels we can pass and vice versa.
                  for link in chosen_shortest_path.keys():
                          gbit_able_to_pass = bottleneck
                          gbit_not_able_to_pass = self.input_traffic - bottleneck

                          self.channel_able_to_pass = math.ceil(gbit_able_to_pass / self.data_rate)

                          # For replacing the old input traffic with new bottleneck as new traffic for calculating FR and ...
                          self.input_traffic_list.append(bottleneck)
                          self.required_channels_list.append(self.channel_able_to_pass)

                          if self.data_rate ==500:
                            if req_channles_dict[link]>= self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0]:
                                channel_not_able_to_pass = req_channles_dict[link] - self.channel_able_to_pass
                            else:
                                channel_not_able_to_pass = 0
                          else:
                            if req_channles_dict[link]>= self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0]:
                                channel_not_able_to_pass = req_channles_dict[link] - self.channel_able_to_pass
                            else:
                                channel_not_able_to_pass = 0

                          print(f'But for this link({link}) we can pass ({self.channel_able_to_pass}) channels with {self.data_rate}Gb/s and we must reserve ({channel_not_able_to_pass}) for the next shortest_path')


                          if self.data_rate ==500:
                            if channel_not_able_to_pass>0:
                                req_channles_dict[link] = channel_not_able_to_pass
                                self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0]= 0
                                self.G[link[0]][link[1]]['channels']['WDM_500'][0] = 0

                            elif channel_not_able_to_pass==0:
                                req_channles_dict[link] = 0
                                self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0]-= self.channel_able_to_pass
                                self.G[link[0]][link[1]]['channels']['WDM_500'][0] -= self.channel_able_to_pass

                          else:

                            if channel_not_able_to_pass>0:
                                req_channles_dict[link] = channel_not_able_to_pass
                                self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0]= 0
                                self.G[link[0]][link[1]]['channels']['WDM_400'][0] = 0

                            elif channel_not_able_to_pass==0:
                                req_channles_dict[link] = 0
                                self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0]-= self.channel_able_to_pass
                                self.G[link[0]][link[1]]['channels']['WDM_400'][0] -= self.channel_able_to_pass

                  print('\nPrevious shortest_path has been updated-->',self.shortest_paths_with_channels[shortest_path_name])
                  print('\nBut we have still some required channels. So, we go through another shortest path: ')
                  self.channel_attribute(source=source,destination=destination)# call channel to update free channels
                  self.input_traffic = gbit_not_able_to_pass # For other part


            #Checking block
            # We set minor block which means one shortest path can be block not all shortest paths
            self.minor_block_500 = 0
            self.minor_block_400 = 0
            for shortest_path_name in self.shortest_paths_with_channels.keys():
                chosen_shortest_path= self.shortest_paths_with_channels[shortest_path_name] # loop for all shortest_paths

                self.total_distance1 = 0
                for link in chosen_shortest_path.keys():
                  self.total_distance1 += self.G[link[0]][link[1]]['weight']

                if self.total_distance1 <= 1300:
                  for link in chosen_shortest_path.keys():

                      if chosen_shortest_path[link]['setup_channels']['WDM_500'][0]==0:
                          self.minor_block_500 +=1
                          break


                elif 1300 < self.total_distance1 :
                  for link in chosen_shortest_path.keys():

                      if chosen_shortest_path[link]['setup_channels']['WDM_400'][0]==0:
                          self.minor_block_400 +=1
                          break


            if (self.minor_block_400 + self.minor_block_500) == len(self.shortest_paths_with_channels.keys()):
                print('Blocking will happen here, and no_of_blocked_demand for this demand is: ',no_demand)
                print('self.minor_block_400',self.minor_block_400)
                print('self.minor_bloc_500',self.minor_block_500)
                self.blocked_demand_list.append(no_demand)

            else:
                print('number of shortest_paths:',len(self.shortest_paths_with_channels.keys()))
                print('self.minor_block_400:',self.minor_block_400)
                print('self.minor_block_500:',self.minor_block_500)
                self.blocked_demand_list.append(0)


        # Calculate all parameters we need for the demand
            self.calculations()
            # no_continue = False

            # if self.BR >1:

            #   self.calculations()

            #   no_continue = True
            #   break


        print('='*200)
        print('50 new random demand added')
        print('='*200)
        # Add another 50 random demands to the demands, avaialble in system
        # if no_continue == False:
        self.updating()


################################################################################
      def updating(self):
          """ Check Blocking < 1 or not, if so, program will be stopped. Otherwise it addes 50 random demands """

          if self.BR<1:

                  self.step +=1
                  self.update_demand =True
                  self.add_new_demand()
                  self.routing()


          else:
            print(f'The network has been stopped because of BlooCking rate {self.BR}')

################################################################################
      def calculations(self):
          """Calculate FR, AVO, BR"""

          self.total_no_of_demands = 0
          self.total_no_of_demands = self.step * 50 + 300
          print(f'So far, total_no_of_demands is :{self.total_no_of_demands}')

          self.no_of_blocked_demand = sum(self.blocked_demand_list)
          self.used_bandwidth = sum(self.input_traffic_list)
          self.used_channels = sum(self.required_channels_list)


          #self.calculation1()
          self.FR = (self.used_bandwidth / self.total_bandwidth)*100
          self.AVO = (self.used_channels / self.total_channels)*100
          self.BR = (self.no_of_blocked_demand / self.total_no_of_demands)*100
          #####
          print('total_no_blocked_demands',self.no_of_blocked_demand)
          print('self.total_bandwidth',self.total_bandwidth)
          print('self.total_channels',self.total_channels)
          print('FR = (self.used_bandwidth / self.total_bandwidth)*100')
          print('AVO = (self.used_channels / self.total_channels)*100')
          print(f'used_bandwidth: {self.used_bandwidth } -->{self.input_traffic_list}')
          print(f'used channels: {self.used_channels} --->{self.required_channels_list}')
          print(f'total_channels: {self.total_channels}')
          print('Filling Ratio is: ',self.FR)
          print('Average Fiber Ocupation: ',self.AVO)
          print('Blocking Rate: ',self.BR)
          self.info_dict[f'step {self.step}'] = {'FR':self.FR,'AVO':self.AVO,'BR':self.BR}

################################################################################
      def calculation1(self):
        self.edge_labels = nx.get_edge_attributes(self.G, 'channels')
        no_of_wdm_500 = 0
        no_of_wdm_400 = 0
        for link in self.edge_labels.keys():
          no_of_wdm_500 += self.edge_labels[link]['WDM_500'][0]
          no_of_wdm_400 += self.edge_labels[link]['WDM_400'][0]

        self.total_bandwidth = no_of_wdm_500 *500 + no_of_wdm_400 *400
        self.total_channels = no_of_wdm_500 + no_of_wdm_400

      def store_values(self):
        """Return values we need"""

        return self.info_dict, self.generated_demands





In [ ]:
CND = Bottom_up_national_without_grooming()
route = CND.routing()

In [23]:
values,generated_demands = CND.store_values()
Bottom_up_national_without_grooming_table = pd.DataFrame(values)
Bottom_up_national_without_grooming_table.to_csv('Bottom_up_national_without_grooming.csv')
Bottom_up_national_without_grooming_table

,step 0,step 1,step 2,step 3,step 4,step 5,step 6,step 7
FR,19.696243,21.795472,24.130106,25.819251,28.745468,31.144279,33.656821,36.336634
AVO,25.579536,43.169399,47.579425,51.947098,57.360406,61.904762,67.651195,73.055163
BR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.230769


# 7: Bottom_up_continental_with_grooming

In [24]:
class Bottom_up_continental_with_grooming():


      def __init__(self):
        self.full_mesh()
        self.update_demand = False
        self.step = 0
        self.info_dict = {}  # to store values

        self.table = pd.read_csv('/content/random_demands.csv')
        self.initial_number = 300
        self.generated_demands = self.table.iloc[:self.initial_number]

      def setup_graph(self):

        global distnace_list_cont
        # Create a complete graph with 8 nodes
        self.G = nx.complete_graph(8)

        # List all possible edges
        all_edges = list(self.G.edges())

        # Remove edges until the number of edges reaches
        while len(self.G.edges()) > 20:
            # Remove the last edge from the list
            edge_to_remove = all_edges.pop()

            # Remove the chosen edge
            self.G.remove_edge(*edge_to_remove)

        # Print the edges of the resulting graph
        self.node_list = self.G.nodes()
        self.edge_list = self.G.edges()

        print('links',len(self.edge_list))

        for index, edge in enumerate(self.G.edges()):
          weight = distnace_list_cont[index]
          self.G.add_edge(*edge,weight=weight,channels={'WDM_500':[0],'WDM_400':[0]})


      def full_mesh(self):
        self.paths = []
        self.setup_graph()
        n = 1
        while n<=100:
          for source in self.node_list:
            for destination in self.node_list:
              if source != destination:
                self.path = (source,destination)
                self.paths.append(self.path)
          n +=1
        # repated this full mesh for 100 times

      def add_new_demand(self):
          """ad 50 another demands to random_demands table"""

          self.initial_number += 50
          self.generated_demands = self.table.iloc[:self.initial_number]
          self.grooming()




      def channel_attribute(self,source,destination):
          """Add channel attribute to each  link of chosen shortest_path"""

          self.source = source
          self.destination = destination

          # call shortest_path
          self.shortest_paths(self.source,self.destination)
          self.sh_p = self.sorted_paths

          self.shortest_paths_with_channels ={}
          for shortest_path_key in self.sh_p.keys(): # loop for all shortest_paths
              chosen_shortest_path = self.sh_p[shortest_path_key]
              no_links_here =len(chosen_shortest_path['path'])-1

              link_attrs ={}
              for i in range(no_links_here):
                channels = self.G[chosen_shortest_path['path'][i]][chosen_shortest_path['path'][i+1]]['channels']
                link_attrs[(chosen_shortest_path['path'][i], chosen_shortest_path['path'][i+1])]={"setup_channels":channels }


              self.shortest_paths_with_channels[shortest_path_key] = link_attrs

################################################################################
      def grooming(self):

          self.new_table = pd.DataFrame({'no_demands':[],'data_rate':[],'source':[],'destination':[]})
          for source in self.node_list:
            for destination in self.node_list:
              if source != destination:
                for data_rate  in [100,400] :
                  result = self.generated_demands[((self.generated_demands.source==source) & (self.generated_demands.destination==destination) & (self.generated_demands.data_rate==data_rate))]
                  no_demands = result.shape[0]
                  self.new_table.loc[len(self.new_table)] = [no_demands,data_rate,source,destination]

          self.generated_demands = self.new_table
          print('I am in grooming')
################################################################################
      def set_wdm(self):
          for path in self.paths:
              #print(f'path is like this: {path}')
              source = path[0]
              destination = path[1]
              # then set channel attributes to these paths
              self.channel_attribute(source=source, destination=destination)

              counter = 0
              for shortest_path_name in self.shortest_paths_with_channels.keys():  # loop for all shortest_paths
                  chosen_shortest_path = self.shortest_paths_with_channels[shortest_path_name]
                  #print('\nFor this path, chosed shortest path is--->', chosen_shortest_path)

                  positive_counter = 0
                  req_channles_dict = {}

                  for link in chosen_shortest_path.keys():

                      number_of_wdms_500 = chosen_shortest_path[link]['setup_channels']['WDM_500'][0]
                      number_of_wdms_400 = chosen_shortest_path[link]['setup_channels']['WDM_400'][0]
                      number_of_wdms = number_of_wdms_500 + number_of_wdms_400

                      self.setup_channels = number_of_wdms

                      if self.setup_channels < 96:

                          #print(f'yes {self.setup_channels}({number_of_wdms_500} + {number_of_wdms_400}) <96')
                          positive_counter += 1

                  #print(f'positive_counter : {positive_counter}')

                  self.total_distance = 0
                  for link in chosen_shortest_path.keys():
                      self.total_distance += self.G[link[0]][link[1]]['weight']

                  if self.total_distance <= 1300:
                      #print(f'we must set 1 WDM with 500 Gb/s since distance <=1300')
                      self.WDM_data_rate = 500

                  elif 1300 < self.total_distance <= 2500:
                      #print(f'we must set 1 WDM with 400 Gb/s since 1300 <distance <=2500')
                      self.WDM_data_rate = 400
                  else:
                      self.next_shortest_path = True

                  # capacity to pass the demand
                  if positive_counter == len(chosen_shortest_path.keys()):

                      # pass required_channels from each link of chosen path and update free channels
                      for link in chosen_shortest_path.keys():
                          if self.WDM_data_rate == 500:
                            old_wdm_500 = self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0]
                            self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0] = old_wdm_500 + 1
                          else:
                            old_wdm_400 = self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0]
                            self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0] = old_wdm_400 + 1
                          # self.G[link[0]][link[1]]['channels'].append(self.WDM_data_rate)

                      #print('\nUpdataed chosed_shortest_path is: ', self.shortest_paths_with_channels[shortest_path_name])
                      #print('=' * 200)

                      # Here, we use break, since the chosen pass could completely pass the demand and there is no need
                      # to check other short paths for this demand
                      break
                  counter += 1

              if counter == len(self.shortest_paths_with_channels.keys()):
                  #print('there is no space to setup wdm')
                  #self.draw_graph()
                  self.calculation1()
                  break





################################################################################
      def draw_graph(self,title='now'):
          """draw_graph function"""

          nx.draw(self.G,with_labels=True,node_color=self.node_color,pos=self.node_positions)
          self.edge_labels = nx.get_edge_attributes(self.G, 'channels')
          plt.title(title, fontweight="bold")
          nx.draw_networkx_edge_labels(self.G, pos=self.node_positions, edge_labels=self.edge_labels)
          plt.show()
          print(self.edge_labels)

################################################################################
      def shortest_paths(self,source,destination):
          """Find all shortest paths for each source and destination"""

          self.source = source
          self.destination = destination
          paths_dict ={}
          # We use simple paths at the beginning
          all_simple_paths = list(nx.all_simple_paths(self.G, self.source, self.destination)) # l

          index = 1
          for path in all_simple_paths:
            weight = sum(self.G[path[i]][path[i+1]]['weight'] for i in range(len(path)-1))
            paths_dict[f'path_{index}'] = {'path':path,'weight':weight,'setup_channels':self.G[path[0]][path[1]]['channels'] }
            index+=1

          # Sort all simple paths based on their weight to get shortest_paths
          self.sorted_paths = dict(sorted(paths_dict.items(), key=lambda item: item[1]['weight']))

################################################################################
      def routing(self):

        # set graph and wdm
        self.set_wdm()


        # check if there are initial demands in the system or not. If so, we do not need to call initial_random_demand
        if self.update_demand ==False:
          self.initial_number = 300
          self.generated_demands = self.table.iloc[:self.initial_number]
          self.grooming()

        else:
          pass


        self.input_traffic_list = []
        self.required_channels_list =[]
        self.blocked_demand_list = []


        # iterate over all demands
        #for demand in self.generated_demands:
        jj = 0
        for index in self.generated_demands.index:
            demand = self.generated_demands.iloc[index]


            no_demand = demand['no_demands']
            data_rate = demand['data_rate']
            source = demand['source']
            destination = demand['destination']
            print('*'*100)
            print(f'\t\t\t\t\t ******  Demand {index} at this level is [{no_demand},{data_rate},({source},{destination})]  ******')

            # calculate input traffic
            self.input_traffic = no_demand*data_rate

            # call channel_attribute to first find short paths between source and destination,
            # then set channel attributes to these paths
            self.channel_attribute(source=source,destination=destination)


            # iterate over all short paths available for the demand
            for shortest_path_name in self.shortest_paths_with_channels.keys():

                # Chose one shortest path
                chosen_shortest_path= self.shortest_paths_with_channels[shortest_path_name] # loop for all shortest_paths
                print('\nFor this demand, chosed shortest path is--->',chosen_shortest_path)

                positive_counter_500 = 0
                positive_counter_400 = 0
                req_channles_dict = {}


                self.total_distance = 0
                for link in chosen_shortest_path.keys():
                  self.total_distance += self.G[link[0]][link[1]]['weight']


                if self.total_distance <=1300:
                  print(f'Set data_rate to 500, since total_distance <= 1300')
                  self.data_rate=500

                elif 1300 < self.total_distance <= 2500:
                  print(f'Set data_rate to 400, since 1300 < total_distan <= 2500')
                  self.data_rate=400
                else:
                  self.next_shortest_path = True


                # iterate over all links of chosen the shortest path
                for link in chosen_shortest_path.keys():

                      self.free_channel_500 = chosen_shortest_path[link]['setup_channels']['WDM_500'][0]
                      self.free_channel_400 = chosen_shortest_path[link]['setup_channels']['WDM_400'][0]


                      # add input traffic of this link to input_traffic_list to use in FR, AVO and ...
                      self.input_traffic_list.append(self.input_traffic)

                      # calculate the number WDM trails(required_channels) and add it to required_channels_list
                      self.required_channels = math.ceil(self.input_traffic / self.data_rate)

                      self.required_channels_list.append(self.required_channels)
                      req_channles_dict[link] = self.required_channels


                      if self.data_rate == 500  and self.free_channel_500 >= self.required_channels:
                            positive_counter_500 +=1

                      elif self.data_rate == 400  and self.free_channel_400 >= self.required_channels:
                            positive_counter_400 +=1


                print(f'Required channels(WDMS) for chosed path is:{(self.input_traffic / self.data_rate)}-->{self.required_channels}')



                # If positive_counter == number of links in a chosen short path, it means all links have appropriate
                # capacity to pass the demand
                if positive_counter_500 == len(chosen_shortest_path.keys()):


                    # pass required_channels from each link of chosen path and update free channels
                    for link in chosen_shortest_path.keys():
                            self.required_channels = req_channles_dict[link]
                            self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0]-= self.required_channels
                            #self.G[link[0]][link[1]]['channels']['WDM_500'][0] -= self.required_channels
                    print('\nUpdataed chosed_shortest_path is: ',self.shortest_paths_with_channels[shortest_path_name])
                    print('='*200)


                    # Here, we use break, since the chosen pass could completely pass the demand and there is no need
                    # to check other short paths for this demand
                    break

                elif positive_counter_400 == len(chosen_shortest_path.keys()):


                    # pass required_channels from each link of chosen path and update free channels
                    for link in chosen_shortest_path.keys():
                            self.required_channels = req_channles_dict[link]
                            self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0]-= self.required_channels
                            #self.G[link[0]][link[1]]['channels']['WDM_400'][0] -= self.required_channels
                    print('\nUpdataed chosed_shortest_path is: ',self.shortest_paths_with_channels[shortest_path_name])



                    # Here, we use break, since the chosen pass could completely pass the demand and there is no need
                    # to check other short paths for this demand
                    break


                # If positive_counter_500 or 400 != number of links in a chosen short path, it means some links do not
                # have appropriate capacity to pass the demand. So, we need to find bottleneck of that path
                # and pass demand == bottleneck from that chosen path and reserve the rest for next chosen_path
                else:

                  if  self.data_rate ==400:
                    # Find bottleneck. First initial bottleneck
                    bottleneck = 96*400

                    # look at each link . If capacity of that link < bottleneck, we consider it as the new_bottleneck
                    for link in chosen_shortest_path.keys():
                            current_bottleneck = self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0] * self.data_rate

                            if current_bottleneck < bottleneck:
                                bottleneck = current_bottleneck
                  else:
                    bottleneck = 96*500

                    # look at each link . If capacity of that link < bottleneck, we consider it as the new_bottleneck
                    for link in chosen_shortest_path.keys():
                            current_bottleneck = self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0] * self.data_rate

                            if current_bottleneck < bottleneck:
                                bottleneck = current_bottleneck


                  # This part remove some input traffic from list in previous
                  for link in chosen_shortest_path.keys() :
                          self.input_traffic_list.pop()
                          self.required_channels_list.pop()




                  # Iterate over each link to see the number of channels we can pass and vice versa.
                  for link in chosen_shortest_path.keys():
                          gbit_able_to_pass = bottleneck
                          gbit_not_able_to_pass = self.input_traffic - bottleneck

                          self.channel_able_to_pass = math.ceil(gbit_able_to_pass / self.data_rate)

                          # For replacing the old input traffic with new bottleneck as new traffic for calculating FR and ...
                          self.input_traffic_list.append(bottleneck)
                          self.required_channels_list.append(self.channel_able_to_pass)

                          if self.data_rate ==500:
                            if req_channles_dict[link]>= self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0]:
                                channel_not_able_to_pass = req_channles_dict[link] - self.channel_able_to_pass
                            else:
                                channel_not_able_to_pass = 0
                          else:
                            if req_channles_dict[link]>= self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0]:
                                channel_not_able_to_pass = req_channles_dict[link] - self.channel_able_to_pass
                            else:
                                channel_not_able_to_pass = 0

                          print(f'But for this link({link}) we can pass ({self.channel_able_to_pass}) channels with {self.data_rate}Gb/s and we must reserve ({channel_not_able_to_pass}) for the next shortest_path')


                          if self.data_rate ==500:
                            if channel_not_able_to_pass>0:
                                req_channles_dict[link] = channel_not_able_to_pass
                                self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0]= 0
                                self.G[link[0]][link[1]]['channels']['WDM_500'][0] = 0

                            elif channel_not_able_to_pass==0:
                                req_channles_dict[link] = 0
                                self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0]-= self.channel_able_to_pass
                                self.G[link[0]][link[1]]['channels']['WDM_500'][0] -= self.channel_able_to_pass

                          else:

                            if channel_not_able_to_pass>0:
                                req_channles_dict[link] = channel_not_able_to_pass
                                self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0]= 0
                                self.G[link[0]][link[1]]['channels']['WDM_400'][0] = 0

                            elif channel_not_able_to_pass==0:
                                req_channles_dict[link] = 0
                                self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0]-= self.channel_able_to_pass
                                self.G[link[0]][link[1]]['channels']['WDM_400'][0] -= self.channel_able_to_pass

                  print('\nPrevious shortest_path has been updated-->',self.shortest_paths_with_channels[shortest_path_name])
                  print('\nBut we have still some required channels. So, we go through another shortest path: ')
                  self.channel_attribute(source=source,destination=destination)# call channel to update free channels
                  self.input_traffic = gbit_not_able_to_pass # For other part


            #Checking block
            # We set minor block which means one shortest path can be block not all shortest paths
            self.minor_block_500 = 0
            self.minor_block_400 = 0
            for shortest_path_name in self.shortest_paths_with_channels.keys():
                chosen_shortest_path= self.shortest_paths_with_channels[shortest_path_name] # loop for all shortest_paths

                self.total_distance1 = 0
                for link in chosen_shortest_path.keys():
                  self.total_distance1 += self.G[link[0]][link[1]]['weight']

                if self.total_distance1 <= 1300:
                  for link in chosen_shortest_path.keys():

                      if chosen_shortest_path[link]['setup_channels']['WDM_500'][0]==0:
                          self.minor_block_500 +=1
                          break


                elif 1300 < self.total_distance1 :
                  for link in chosen_shortest_path.keys():

                      if chosen_shortest_path[link]['setup_channels']['WDM_400'][0]==0:
                          self.minor_block_400 +=1
                          break


            if (self.minor_block_400 + self.minor_block_500) == len(self.shortest_paths_with_channels.keys()):
                print('Blocking will happen here, and no_of_blocked_demand for this demand is: ',no_demand)
                print('self.minor_block_400',self.minor_block_400)
                print('self.minor_bloc_500',self.minor_block_500)
                self.blocked_demand_list.append(no_demand)

            else:
                print('number of shortest_paths:',len(self.shortest_paths_with_channels.keys()))
                print('self.minor_block_400:',self.minor_block_400)
                print('self.minor_block_500:',self.minor_block_500)
                self.blocked_demand_list.append(0)


        # Calculate all parameters we need for the demand
            self.calculations()
            # no_continue = False

            # if self.BR >1:

            #   self.calculations()

            #   no_continue = True
            #   break


        print('='*200)
        print('50 new random demand added')
        print('='*200)
        # Add another 50 random demands to the demands, avaialble in system
        # if no_continue == False:
        self.updating()


################################################################################
      def updating(self):
          """ Check Blocking < 1 or not, if so, program will be stopped. Otherwise it addes 50 random demands """

          if self.BR<1:

                  self.step +=1
                  self.update_demand =True
                  self.add_new_demand()
                  self.routing()


          else:
            print(f'The network has been stopped because of BlooCking rate {self.BR}')

################################################################################
      def calculations(self):
          """Calculate FR, AVO, BR"""

          self.total_no_of_demands = 0
          self.total_no_of_demands = self.step * 50 + 300
          print(f'So far, total_no_of_demands is :{self.total_no_of_demands}')

          self.no_of_blocked_demand = sum(self.blocked_demand_list)
          self.used_bandwidth = sum(self.input_traffic_list)
          self.used_channels = sum(self.required_channels_list)


          #self.calculation1()
          self.FR = (self.used_bandwidth / self.total_bandwidth)*100
          self.AVO = (self.used_channels / self.total_channels)*100
          self.BR = (self.no_of_blocked_demand / self.total_no_of_demands)*100
          #####
          print('total_no_blocked_demands',self.no_of_blocked_demand)
          print('self.total_bandwidth',self.total_bandwidth)
          print('self.total_channels',self.total_channels)
          print('FR = (self.used_bandwidth / self.total_bandwidth)*100')
          print('AVO = (self.used_channels / self.total_channels)*100')
          print(f'used_bandwidth: {self.used_bandwidth } -->{self.input_traffic_list}')
          print(f'used channels: {self.used_channels} --->{self.required_channels_list}')
          print(f'total_channels: {self.total_channels}')
          print('Filling Ratio is: ',self.FR)
          print('Average Fiber Ocupation: ',self.AVO)
          print('Blocking Rate: ',self.BR)
          self.info_dict[f'step {self.step}'] = {'FR':self.FR,'AVO':self.AVO,'BR':self.BR}

################################################################################
      def calculation1(self):
        self.edge_labels = nx.get_edge_attributes(self.G, 'channels')
        no_of_wdm_500 = 0
        no_of_wdm_400 = 0
        for link in self.edge_labels.keys():
          no_of_wdm_500 += self.edge_labels[link]['WDM_500'][0]
          no_of_wdm_400 += self.edge_labels[link]['WDM_400'][0]

        self.total_bandwidth = no_of_wdm_500 *500 + no_of_wdm_400 *400
        self.total_channels = no_of_wdm_500 + no_of_wdm_400

      def store_values(self):
        """Return values we need"""

        return self.info_dict, self.generated_demands





In [ ]:
CND = Bottom_up_continental_with_grooming()
route = CND.routing()

In [26]:
values,generated_demands = CND.store_values()
Bottom_up_continental_with_grooming_table = pd.DataFrame(values)
Bottom_up_continental_with_grooming_table.to_csv('Bottom_up_continental_with_grooming.csv')
Bottom_up_continental_with_grooming_table

,step 0,step 1,step 2,step 3,step 4,step 5,step 6,step 7,step 8,step 9,step 10,step 11
FR,17.250755,19.465082,21.676301,23.456615,25.988780,28.507358,30.581379,33.365664,33.899644,37.091106,40.250620,46.501000
AVO,22.432024,24.442793,26.878613,28.449502,30.995792,33.496847,34.882108,37.326870,37.738330,41.160594,44.711538,51.210826
BR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.352941


# 8: Bottom_up_continental_without_grooming

In [27]:
class Bottom_up_continental_without_grooming():


      def __init__(self):
        self.full_mesh()
        self.update_demand = False
        self.step = 0
        self.info_dict = {}  # to store values

        self.table = pd.read_csv('/content/random_demands.csv')
        self.initial_number = 300
        self.generated_demands = self.table.iloc[:self.initial_number]

      def setup_graph(self):

        global distnace_list_cont
        # Create a complete graph with 8 nodes
        self.G = nx.complete_graph(8)

        # List all possible edges
        all_edges = list(self.G.edges())

        # Remove edges until the number of edges reaches
        while len(self.G.edges()) > 20:
            # Remove the last edge from the list
            edge_to_remove = all_edges.pop()

            # Remove the chosen edge
            self.G.remove_edge(*edge_to_remove)

        # Print the edges of the resulting graph
        self.node_list = self.G.nodes()
        self.edge_list = self.G.edges()

        print('links',len(self.edge_list))

        for index, edge in enumerate(self.G.edges()):
          weight = distnace_list_cont[index]
          self.G.add_edge(*edge,weight=weight,channels={'WDM_500':[0],'WDM_400':[0]})


      def full_mesh(self):
        self.paths = []
        self.setup_graph()
        n = 1
        while n<=100:
          for source in self.node_list:
            for destination in self.node_list:
              if source != destination:
                self.path = (source,destination)
                self.paths.append(self.path)
          n +=1
        # repated this full mesh for 100 times

      def add_new_demand(self):
          """ad 50 another demands to random_demands table"""

          self.initial_number += 50
          self.generated_demands = self.table.iloc[:self.initial_number]
          #self.grooming()




      def channel_attribute(self,source,destination):
          """Add channel attribute to each  link of chosen shortest_path"""

          self.source = source
          self.destination = destination

          # call shortest_path
          self.shortest_paths(self.source,self.destination)
          self.sh_p = self.sorted_paths

          self.shortest_paths_with_channels ={}
          for shortest_path_key in self.sh_p.keys(): # loop for all shortest_paths
              chosen_shortest_path = self.sh_p[shortest_path_key]
              no_links_here =len(chosen_shortest_path['path'])-1

              link_attrs ={}
              for i in range(no_links_here):
                channels = self.G[chosen_shortest_path['path'][i]][chosen_shortest_path['path'][i+1]]['channels']
                link_attrs[(chosen_shortest_path['path'][i], chosen_shortest_path['path'][i+1])]={"setup_channels":channels }


              self.shortest_paths_with_channels[shortest_path_key] = link_attrs

################################################################################
      def grooming(self):

          self.new_table = pd.DataFrame({'no_demands':[],'data_rate':[],'source':[],'destination':[]})
          for source in self.node_list:
            for destination in self.node_list:
              if source != destination:
                for data_rate  in [100,400] :
                  result = self.generated_demands[((self.generated_demands.source==source) & (self.generated_demands.destination==destination) & (self.generated_demands.data_rate==data_rate))]
                  no_demands = result.shape[0]
                  self.new_table.loc[len(self.new_table)] = [no_demands,data_rate,source,destination]

          self.generated_demands = self.new_table
          print('I am in grooming')
################################################################################
      def set_wdm(self):
          for path in self.paths:
              #print(f'path is like this: {path}')
              source = path[0]
              destination = path[1]
              # then set channel attributes to these paths
              self.channel_attribute(source=source, destination=destination)

              counter = 0
              for shortest_path_name in self.shortest_paths_with_channels.keys():  # loop for all shortest_paths
                  chosen_shortest_path = self.shortest_paths_with_channels[shortest_path_name]
                  #print('\nFor this path, chosed shortest path is--->', chosen_shortest_path)

                  positive_counter = 0
                  req_channles_dict = {}

                  for link in chosen_shortest_path.keys():

                      number_of_wdms_500 = chosen_shortest_path[link]['setup_channels']['WDM_500'][0]
                      number_of_wdms_400 = chosen_shortest_path[link]['setup_channels']['WDM_400'][0]
                      number_of_wdms = number_of_wdms_500 + number_of_wdms_400

                      self.setup_channels = number_of_wdms

                      if self.setup_channels < 96:

                          #print(f'yes {self.setup_channels}({number_of_wdms_500} + {number_of_wdms_400}) <96')
                          positive_counter += 1

                  #print(f'positive_counter : {positive_counter}')

                  self.total_distance = 0
                  for link in chosen_shortest_path.keys():
                      self.total_distance += self.G[link[0]][link[1]]['weight']

                  if self.total_distance <= 1300:
                      #print(f'we must set 1 WDM with 500 Gb/s since distance <=1300')
                      self.WDM_data_rate = 500

                  elif 1300 < self.total_distance <= 2500:
                      #print(f'we must set 1 WDM with 400 Gb/s since 1300 <distance <=2500')
                      self.WDM_data_rate = 400
                  else:
                      self.next_shortest_path = True

                  # capacity to pass the demand
                  if positive_counter == len(chosen_shortest_path.keys()):

                      # pass required_channels from each link of chosen path and update free channels
                      for link in chosen_shortest_path.keys():
                          if self.WDM_data_rate == 500:
                            old_wdm_500 = self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0]
                            self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0] = old_wdm_500 + 1
                          else:
                            old_wdm_400 = self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0]
                            self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0] = old_wdm_400 + 1
                          # self.G[link[0]][link[1]]['channels'].append(self.WDM_data_rate)

                      #print('\nUpdataed chosed_shortest_path is: ', self.shortest_paths_with_channels[shortest_path_name])
                      #print('=' * 200)

                      # Here, we use break, since the chosen pass could completely pass the demand and there is no need
                      # to check other short paths for this demand
                      break
                  counter += 1

              if counter == len(self.shortest_paths_with_channels.keys()):
                  #print('there is no space to setup wdm')
                  #self.draw_graph()
                  self.calculation1()
                  break





################################################################################
      def draw_graph(self,title='now'):
          """draw_graph function"""

          nx.draw(self.G,with_labels=True,node_color=self.node_color,pos=self.node_positions)
          self.edge_labels = nx.get_edge_attributes(self.G, 'channels')
          plt.title(title, fontweight="bold")
          nx.draw_networkx_edge_labels(self.G, pos=self.node_positions, edge_labels=self.edge_labels)
          plt.show()
          print(self.edge_labels)

################################################################################
      def shortest_paths(self,source,destination):
          """Find all shortest paths for each source and destination"""

          self.source = source
          self.destination = destination
          paths_dict ={}
          # We use simple paths at the beginning
          all_simple_paths = list(nx.all_simple_paths(self.G, self.source, self.destination)) # l

          index = 1
          for path in all_simple_paths:
            weight = sum(self.G[path[i]][path[i+1]]['weight'] for i in range(len(path)-1))
            paths_dict[f'path_{index}'] = {'path':path,'weight':weight,'setup_channels':self.G[path[0]][path[1]]['channels'] }
            index+=1

          # Sort all simple paths based on their weight to get shortest_paths
          self.sorted_paths = dict(sorted(paths_dict.items(), key=lambda item: item[1]['weight']))

################################################################################
      def routing(self):

        # set graph and wdm
        self.set_wdm()


        # check if there are initial demands in the system or not. If so, we do not need to call initial_random_demand
        if self.update_demand ==False:
          self.initial_number = 300
          self.generated_demands = self.table.iloc[:self.initial_number]
          self.grooming()

        else:
          pass


        self.input_traffic_list = []
        self.required_channels_list =[]
        self.blocked_demand_list = []


        # iterate over all demands
        #for demand in self.generated_demands:
        jj = 0
        for index in self.generated_demands.index:
            demand = self.generated_demands.iloc[index]


            no_demand = demand['no_demands']
            data_rate = demand['data_rate']
            source = demand['source']
            destination = demand['destination']
            print('*'*100)
            print(f'\t\t\t\t\t ******  Demand {index} at this level is [{no_demand},{data_rate},({source},{destination})]  ******')

            # calculate input traffic
            self.input_traffic = no_demand*data_rate

            # call channel_attribute to first find short paths between source and destination,
            # then set channel attributes to these paths
            self.channel_attribute(source=source,destination=destination)


            # iterate over all short paths available for the demand
            for shortest_path_name in self.shortest_paths_with_channels.keys():

                # Chose one shortest path
                chosen_shortest_path= self.shortest_paths_with_channels[shortest_path_name] # loop for all shortest_paths
                print('\nFor this demand, chosed shortest path is--->',chosen_shortest_path)

                positive_counter_500 = 0
                positive_counter_400 = 0
                req_channles_dict = {}


                self.total_distance = 0
                for link in chosen_shortest_path.keys():
                  self.total_distance += self.G[link[0]][link[1]]['weight']


                if self.total_distance <=1300:
                  print(f'Set data_rate to 500, since total_distance <= 1300')
                  self.data_rate=500

                elif 1300 < self.total_distance <= 2500:
                  print(f'Set data_rate to 400, since 1300 < total_distan <= 2500')
                  self.data_rate=400
                else:
                  self.next_shortest_path = True


                # iterate over all links of chosen the shortest path
                for link in chosen_shortest_path.keys():

                      self.free_channel_500 = chosen_shortest_path[link]['setup_channels']['WDM_500'][0]
                      self.free_channel_400 = chosen_shortest_path[link]['setup_channels']['WDM_400'][0]


                      # add input traffic of this link to input_traffic_list to use in FR, AVO and ...
                      self.input_traffic_list.append(self.input_traffic)

                      # calculate the number WDM trails(required_channels) and add it to required_channels_list
                      self.required_channels = math.ceil(self.input_traffic / self.data_rate)

                      self.required_channels_list.append(self.required_channels)
                      req_channles_dict[link] = self.required_channels


                      if self.data_rate == 500  and self.free_channel_500 >= self.required_channels:
                            positive_counter_500 +=1

                      elif self.data_rate == 400  and self.free_channel_400 >= self.required_channels:
                            positive_counter_400 +=1


                print(f'Required channels(WDMS) for chosed path is:{(self.input_traffic / self.data_rate)}-->{self.required_channels}')



                # If positive_counter == number of links in a chosen short path, it means all links have appropriate
                # capacity to pass the demand
                if positive_counter_500 == len(chosen_shortest_path.keys()):


                    # pass required_channels from each link of chosen path and update free channels
                    for link in chosen_shortest_path.keys():
                            self.required_channels = req_channles_dict[link]
                            self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0]-= self.required_channels
                            #self.G[link[0]][link[1]]['channels']['WDM_500'][0] -= self.required_channels
                    print('\nUpdataed chosed_shortest_path is: ',self.shortest_paths_with_channels[shortest_path_name])
                    print('='*200)


                    # Here, we use break, since the chosen pass could completely pass the demand and there is no need
                    # to check other short paths for this demand
                    break

                elif positive_counter_400 == len(chosen_shortest_path.keys()):


                    # pass required_channels from each link of chosen path and update free channels
                    for link in chosen_shortest_path.keys():
                            self.required_channels = req_channles_dict[link]
                            self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0]-= self.required_channels
                            #self.G[link[0]][link[1]]['channels']['WDM_400'][0] -= self.required_channels
                    print('\nUpdataed chosed_shortest_path is: ',self.shortest_paths_with_channels[shortest_path_name])



                    # Here, we use break, since the chosen pass could completely pass the demand and there is no need
                    # to check other short paths for this demand
                    break


                # If positive_counter_500 or 400 != number of links in a chosen short path, it means some links do not
                # have appropriate capacity to pass the demand. So, we need to find bottleneck of that path
                # and pass demand == bottleneck from that chosen path and reserve the rest for next chosen_path
                else:

                  if  self.data_rate ==400:
                    # Find bottleneck. First initial bottleneck
                    bottleneck = 96*400

                    # look at each link . If capacity of that link < bottleneck, we consider it as the new_bottleneck
                    for link in chosen_shortest_path.keys():
                            current_bottleneck = self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0] * self.data_rate

                            if current_bottleneck < bottleneck:
                                bottleneck = current_bottleneck
                  else:
                    bottleneck = 96*500

                    # look at each link . If capacity of that link < bottleneck, we consider it as the new_bottleneck
                    for link in chosen_shortest_path.keys():
                            current_bottleneck = self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0] * self.data_rate

                            if current_bottleneck < bottleneck:
                                bottleneck = current_bottleneck


                  # This part remove some input traffic from list in previous
                  for link in chosen_shortest_path.keys() :
                          self.input_traffic_list.pop()
                          self.required_channels_list.pop()




                  # Iterate over each link to see the number of channels we can pass and vice versa.
                  for link in chosen_shortest_path.keys():
                          gbit_able_to_pass = bottleneck
                          gbit_not_able_to_pass = self.input_traffic - bottleneck

                          self.channel_able_to_pass = math.ceil(gbit_able_to_pass / self.data_rate)

                          # For replacing the old input traffic with new bottleneck as new traffic for calculating FR and ...
                          self.input_traffic_list.append(bottleneck)
                          self.required_channels_list.append(self.channel_able_to_pass)

                          if self.data_rate ==500:
                            if req_channles_dict[link]>= self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0]:
                                channel_not_able_to_pass = req_channles_dict[link] - self.channel_able_to_pass
                            else:
                                channel_not_able_to_pass = 0
                          else:
                            if req_channles_dict[link]>= self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0]:
                                channel_not_able_to_pass = req_channles_dict[link] - self.channel_able_to_pass
                            else:
                                channel_not_able_to_pass = 0

                          print(f'But for this link({link}) we can pass ({self.channel_able_to_pass}) channels with {self.data_rate}Gb/s and we must reserve ({channel_not_able_to_pass}) for the next shortest_path')


                          if self.data_rate ==500:
                            if channel_not_able_to_pass>0:
                                req_channles_dict[link] = channel_not_able_to_pass
                                self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0]= 0
                                self.G[link[0]][link[1]]['channels']['WDM_500'][0] = 0

                            elif channel_not_able_to_pass==0:
                                req_channles_dict[link] = 0
                                self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_500'][0]-= self.channel_able_to_pass
                                self.G[link[0]][link[1]]['channels']['WDM_500'][0] -= self.channel_able_to_pass

                          else:

                            if channel_not_able_to_pass>0:
                                req_channles_dict[link] = channel_not_able_to_pass
                                self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0]= 0
                                self.G[link[0]][link[1]]['channels']['WDM_400'][0] = 0

                            elif channel_not_able_to_pass==0:
                                req_channles_dict[link] = 0
                                self.shortest_paths_with_channels[shortest_path_name][link]['setup_channels']['WDM_400'][0]-= self.channel_able_to_pass
                                self.G[link[0]][link[1]]['channels']['WDM_400'][0] -= self.channel_able_to_pass

                  print('\nPrevious shortest_path has been updated-->',self.shortest_paths_with_channels[shortest_path_name])
                  print('\nBut we have still some required channels. So, we go through another shortest path: ')
                  self.channel_attribute(source=source,destination=destination)# call channel to update free channels
                  self.input_traffic = gbit_not_able_to_pass # For other part


            #Checking block
            # We set minor block which means one shortest path can be block not all shortest paths
            self.minor_block_500 = 0
            self.minor_block_400 = 0
            for shortest_path_name in self.shortest_paths_with_channels.keys():
                chosen_shortest_path= self.shortest_paths_with_channels[shortest_path_name] # loop for all shortest_paths

                self.total_distance1 = 0
                for link in chosen_shortest_path.keys():
                  self.total_distance1 += self.G[link[0]][link[1]]['weight']

                if self.total_distance1 <= 1300:
                  for link in chosen_shortest_path.keys():

                      if chosen_shortest_path[link]['setup_channels']['WDM_500'][0]==0:
                          self.minor_block_500 +=1
                          break


                elif 1300 < self.total_distance1 :
                  for link in chosen_shortest_path.keys():

                      if chosen_shortest_path[link]['setup_channels']['WDM_400'][0]==0:
                          self.minor_block_400 +=1
                          break


            if (self.minor_block_400 + self.minor_block_500) == len(self.shortest_paths_with_channels.keys()):
                print('Blocking will happen here, and no_of_blocked_demand for this demand is: ',no_demand)
                print('self.minor_block_400',self.minor_block_400)
                print('self.minor_bloc_500',self.minor_block_500)
                self.blocked_demand_list.append(no_demand)

            else:
                print('number of shortest_paths:',len(self.shortest_paths_with_channels.keys()))
                print('self.minor_block_400:',self.minor_block_400)
                print('self.minor_block_500:',self.minor_block_500)
                self.blocked_demand_list.append(0)


        # Calculate all parameters we need for the demand
            self.calculations()
            # no_continue = False

            # if self.BR >1:

            #   self.calculations()

            #   no_continue = True
            #   break


        print('='*200)
        print('50 new random demand added')
        print('='*200)
        # Add another 50 random demands to the demands, avaialble in system
        # if no_continue == False:
        self.updating()


################################################################################
      def updating(self):
          """ Check Blocking < 1 or not, if so, program will be stopped. Otherwise it addes 50 random demands """

          if self.BR<1:

                  self.step +=1
                  self.update_demand =True
                  self.add_new_demand()
                  self.routing()


          else:
            print(f'The network has been stopped because of BlooCking rate {self.BR}')

################################################################################
      def calculations(self):
          """Calculate FR, AVO, BR"""

          self.total_no_of_demands = 0
          self.total_no_of_demands = self.step * 50 + 300
          print(f'So far, total_no_of_demands is :{self.total_no_of_demands}')

          self.no_of_blocked_demand = sum(self.blocked_demand_list)
          self.used_bandwidth = sum(self.input_traffic_list)
          self.used_channels = sum(self.required_channels_list)


          #self.calculation1()
          self.FR = (self.used_bandwidth / self.total_bandwidth)*100
          self.AVO = (self.used_channels / self.total_channels)*100
          self.BR = (self.no_of_blocked_demand / self.total_no_of_demands)*100
          #####
          print('total_no_blocked_demands',self.no_of_blocked_demand)
          print('self.total_bandwidth',self.total_bandwidth)
          print('self.total_channels',self.total_channels)
          print('FR = (self.used_bandwidth / self.total_bandwidth)*100')
          print('AVO = (self.used_channels / self.total_channels)*100')
          print(f'used_bandwidth: {self.used_bandwidth } -->{self.input_traffic_list}')
          print(f'used channels: {self.used_channels} --->{self.required_channels_list}')
          print(f'total_channels: {self.total_channels}')
          print('Filling Ratio is: ',self.FR)
          print('Average Fiber Ocupation: ',self.AVO)
          print('Blocking Rate: ',self.BR)
          self.info_dict[f'step {self.step}'] = {'FR':self.FR,'AVO':self.AVO,'BR':self.BR}

################################################################################
      def calculation1(self):
        self.edge_labels = nx.get_edge_attributes(self.G, 'channels')
        no_of_wdm_500 = 0
        no_of_wdm_400 = 0
        for link in self.edge_labels.keys():
          no_of_wdm_500 += self.edge_labels[link]['WDM_500'][0]
          no_of_wdm_400 += self.edge_labels[link]['WDM_400'][0]

        self.total_bandwidth = no_of_wdm_500 *500 + no_of_wdm_400 *400
        self.total_channels = no_of_wdm_500 + no_of_wdm_400

      def store_values(self):
        """Return values we need"""

        return self.info_dict, self.generated_demands





In [ ]:
CND = Bottom_up_continental_without_grooming()
route = CND.routing()

In [29]:
values,generated_demands = CND.store_values()
Bottom_up_continental_without_grooming_table = pd.DataFrame(values)
Bottom_up_continental_without_grooming_table.to_csv('Bottom_up_continental_without_grooming.csv')
Bottom_up_continental_without_grooming_table

,step 0,step 1,step 2,step 3,step 4,step 5,step 6,step 7,step 8
FR,17.250755,19.390788,21.926471,23.585714,27.530775,30.905077,33.710879,37.560241,40.721412
AVO,22.432024,38.558692,43.235294,47.428571,54.453295,61.147903,67.511177,75.301205,82.271681
BR,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.153846,1.428571
